# Introduction to SQL


---

## Introduction to Databases

When the data changes frequently, requires shared access, doesn't fit in memory, and security is critical, a database is a great solution. A database is a data representation that lives on disk that can be queried, accessed, and updated without using much memory. We primarily interact with a database using a [database management system](https://en.wikipedia.org/wiki/Database) or **DBMS** for short.

In the pandas workflow, we spend most of our time thinking about what functions and methods to use, where to store intermediate results in variables, and juggling all of these. To work with data stored in a database, we instead use a language called **SQL** (or structured query language). In SQL, we express each unique request (whether it be fetching a subset of or editing values in the data) as a single query and then ask the DBMS to run the query and display any results.

For example, to fetch a specific subset of the data from a database, we would:

* write the SQL query: `SELECT * FROM salaries`
* ask the DBMS to run the query and display the results to us

Because the data lives on disk, we can work with datasets that consume multiple terabytes of disk space. Many data science teams in industry have servers and setups in cloud environments like Microsoft Azure or Amazon Web Services that let team members work with this scale of data. Robust and popular DBMS tools like [Postgres](https://www.postgresql.org/) and [MySQL](https://www.mysql.com/) include powerful features for managing user credentials, security, and high data throughput (quickly changing data). We'll learn the fundamentals of SQL using a small, portable DBMS called [SQLite](https://www.sqlite.org/). SQLite is the most popular database in the world and is lightweight enough that the SQLite DBMS is included as a [module in Python](https://docs.python.org/3.6/library/sqlite3.html). Later we'll dive into production systems like Postgres.

We'll explore data from the American Community Survey on job outcome statistics based on college majors. While the original CSV version can be found on [FiveThirtyEight's Github](https://github.com/fivethirtyeight/data/tree/master/college-majors), we'll be using a slightly modified version of the data that's stored as a database. We'll be working with the bit of data that contains the 2010-2012 data for recent college grads only. We'll learn how to write SQL queries to explore and start to understand the dataset.

## Previewing SQLite Module

We'll be working with the sqlite3 Python module, which was developed to work with [SQLite version 3](https://www.sqlite.org/version3.html).

First we'll need to import the module:

In [1]:
import sqlite3

Once we've imported the module, we connect to the database we want to query using the [`connect()` function](https://docs.python.org/3/library/sqlite3.html#sqlite3.connect). This function requires a single parameter, which is the database we want to connect to. Because the database we're working with exists as a file on disk, we need to pass in the file name.

The `connect()` function returns a [Connection instance](https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection), which maintains the connection to the database we want to work with. When we're connected to a database, SQLite locks the database file and prevents any other processes from connecting to the database simultaneously. The SQLite team made this design decision to keep the database lightweight, and avoid the complexity that arises when multiple processes interact with the same database.

Let's connect the `jobs.db` database.

In [2]:
conn = sqlite3.connect("jobs.db")

To display a table from the database we'll need pandas [read_sql_query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) function. Basic syntax for it looks like this:

```python
import pandas as pd
pd.read_sql_query("SQL QUERY", 
                  "URL" or Connection instance, 
                  index_col = "index")
```

Now let's get back to the SQL syntax.

In [3]:
import pandas as pd

## Previewing A Table Using SELECT

A database usually consists of multiple, related tables of data. Each table contains rows and columns, just like a CSV file. We'll be working with the database file `jobs.db`, which contains a single table named `recent_grads`.

To display the first 5 rows from the recent_grads table, we need to:

* write SQL code that expresses this request
* ask the SQLite DBMS software to run the code and display the results

Like other programming languages, code in SQL has to adhere to a defined structure and vocabulary. To specify that we want to return the first 5 rows from `recent_grads`, we need to run the following SQL query:

```SQL
SELECT * FROM recent_grads LIMIT 5
```

Here's what's returned when the query is run:

In [4]:
pd.read_sql_query("\
SELECT * \
FROM recent_grads \
LIMIT 5",
                  conn, index_col = "index")

,Rank,Major_code,Major,Major_category,Total,Sample_size,Men,Women,ShareWomen,Employed,...,Part_time,Full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th,College_jobs,Non_college_jobs,Low_wage_jobs
index,,,,,,,,,,,,,,,,,,,,,
0,1,2419,PETROLEUM ENGINEERING,Engineering,2339,36,2057,282,0.120564,1976,...,270,1207,37,0.018381,110000,95000,125000,1534,364,193
1,2,2416,MINING AND MINERAL ENGINEERING,Engineering,756,7,679,77,0.101852,640,...,170,388,85,0.117241,75000,55000,90000,350,257,50
2,3,2415,METALLURGICAL ENGINEERING,Engineering,856,3,725,131,0.153037,648,...,133,340,16,0.024096,73000,50000,105000,456,176,0
3,4,2417,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,1258,16,1123,135,0.107313,758,...,150,692,40,0.050125,70000,43000,80000,529,102,0
4,5,2405,CHEMICAL ENGINEERING,Engineering,32260,289,21239,11021,0.341631,25694,...,5180,16697,1672,0.061098,65000,50000,75000,18314,4440,972


In this query, we specified:

* the columns we wanted using `SELECT *`
* the table we wanted to query using `FROM recent_grads`
* the number of rows we wanted using `LIMIT 5`

Here's the logic behind it:

```

   SELECT *         FROM recent_grads       LIMIT 5

Which columns?      From which table?    How many rows?
     All              recent_grads             5
     
     
```

## Filtering Rows Using WHERE

Let's answer these questions using the data we have:
* Which majors had mostly female students? Which ones had mostly male students?
* Which majors had the largest spread (difference) between the 25th and 75th percentile starting salaries?
* Which engineering majors had the highest full time employment rates?

Let's start by focusing on the first question. The SQL workflow revolves around translating the question we want to answer to the subset of data we want from the database. To determine which majors had mostly female students, we want the following subset:

* only the Major column
* only the rows where ShareWomen is greater than 0.5 (corresponding to 50%)

To return only the `Major` column, we need to add the specific column name in the `SELECT` statement part of the query (instead of using the `*` operator to return all columns):

```SQL
SELECT Major FROM recent_grads
```

This will return all of the values in the `Major` column. We can specify multiple columns this way as well and the results table will preserve the order of the columns:

```SQL
SELECT Major, Major_category FROM recent_grads
```

To return only the values where `ShareWomen` is greater than or equal to `0.5`, we need to add a `WHERE` clause:

```SQL
SELECT Major FROM recent_grads
WHERE ShareWomen >= 0.5
```

Finally, we can limit the number of rows returned using `LIMIT`:

```SQL
SELECT Major FROM recent_grads
WHERE ShareWomen >= 0.5
LIMIT 5
```

Running this query will return the following results table:

In [5]:
pd.read_sql_query("\
SELECT Major \
FROM recent_grads \
WHERE ShareWomen >= 0.5 \
LIMIT 5",
                 conn)

,Major
0,ACTUARIAL SCIENCE
1,COMPUTER SCIENCE
2,ENVIRONMENTAL ENGINEERING
3,NURSING
4,INDUSTRIAL PRODUCTION TECHNOLOGIES


While in the `SELECT` part of the query, we express the specific **column** we want, in the `WHERE` part we express the specific **rows** we want. The beauty of SQL is that these can be independent.

Now, let's answer the first question:

```SQL
SELECT Major, ShareWomen 
FROM recent_grads 
WHERE ShareWomen < 0.5
```

In [6]:
pd.read_sql_query("\
SELECT Major, ShareWomen \
FROM recent_grads \
WHERE ShareWomen < 0.5", conn)

,Major,ShareWomen
0,PETROLEUM ENGINEERING,0.120564
1,MINING AND MINERAL ENGINEERING,0.101852
2,METALLURGICAL ENGINEERING,0.153037
3,NAVAL ARCHITECTURE AND MARINE ENGINEERING,0.107313
4,CHEMICAL ENGINEERING,0.341631
5,NUCLEAR ENGINEERING,0.144967
6,ASTRONOMY AND ASTROPHYSICS,0.441356
7,MECHANICAL ENGINEERING,0.139793
8,ELECTRICAL ENGINEERING,0.437847
9,COMPUTER ENGINEERING,0.199413


## Expressing Multiple Filter Criteria Using AND, OR and ()

To filter rows by specific criteria, we need to use the `WHERE` statement. A simple `WHERE` statement requires three things:

* The column we want the database to filter on: `ShareWomen`
* A comparison operator that specifies how we want to compare a value in a column: `>`
* The value we want the database to compare each value to: `0.5`

Here are the comparison operators we can use:

* Less than: `<`
* Less than or equal to: `<=`
* Greater than: `>`
* Greater than or equal to: `>=`
* Equal to: `=`
* Not equal to: `!=`

The comparison value after the operator must be either text or a number, depending on the field. Because ShareWomen is a numeric column, we don't need to enclose the number `0.5` in quotes. Finally, **most database systems require that the `SELECT` and `FROM` statements come first, before `WHERE` or any other statements**.

We can use the `AND` operator to combine multiple filter criteria. For example, to determine which engineering majors had majority female, we'd need to specify 2 filtering criteria:

```SQL
SELECT Major 
FROM recent_grads
WHERE Major_category = 'Engineering' AND ShareWomen > 0.5
```

In [7]:
pd.read_sql_query("\
SELECT Major \
FROM recent_grads \
WHERE Major_category = 'Engineering' AND ShareWomen > 0.5",
                 conn)

,Major
0,ENVIRONMENTAL ENGINEERING
1,INDUSTRIAL PRODUCTION TECHNOLOGIES


It looks like only 2 majors met this criteria. If we wanted to "zoom" back out to look at all of the columns for both of these majors to see if they shared some other common attributes, we can modify the `SELECT` statement and use the symbol `*` to represent all columns:

```SQL
SELECT *
FROM recent_grads WHERE Major_category = 'Engineering' AND ShareWomen > 0.5
```

In [8]:
pd.read_sql_query("\
SELECT * \
FROM recent_grads WHERE Major_category = 'Engineering' AND ShareWomen > 0.5",
                 conn,
                 index_col = "index")

,Rank,Major_code,Major,Major_category,Total,Sample_size,Men,Women,ShareWomen,Employed,...,Part_time,Full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th,College_jobs,Non_college_jobs,Low_wage_jobs
index,,,,,,,,,,,,,,,,,,,,,
30,31,2410,ENVIRONMENTAL ENGINEERING,Engineering,4047,26,2639,3339,0.558548,2983,...,930,1951,308,0.093589,50000,42000,56000,2028,830,260
38,39,2503,INDUSTRIAL PRODUCTION TECHNOLOGIES,Engineering,4631,73,528,1588,0.750473,4428,...,597,3242,129,0.028308,46000,35000,65000,1394,2454,480


The ability to quickly iterate on queries as you think of new questions is the appeal of SQL. The SQL workflow lets data professionals focus on asking and answering questions, instead of lower level programming concepts. There's a clear separation of concerns between the engine that stores, organizes, and retrieves the data and the language that let's people interface with the data easily without having to worry about the underlying mechanics.

As the scale of data has increased, engineers have maintained the interface of SQL while swapping out the database engine underneath. This allows people who need to ask and answer questions easily transfer their SQL experience, even as database technologies change. For example, the [Presto project](https://en.wikipedia.org/wiki/Presto_%28SQL_query_engine%29) lets you query using SQL but use data from database systems like MySQL, from a distributed file system like HDFS, and more.

Here's the `AND` example:

```SQL
SELECT Major, Major_category, Median, ShareWomen
FROM recent_grads
WHERE ShareWomen > 0.5 AND Median > 50000
```

In [9]:
# Example with AND operator
pd.read_sql_query("\
SELECT Major, Major_category, Median, ShareWomen \
FROM recent_grads \
WHERE ShareWomen > 0.5 AND Median > 50000",
                 conn)

,Major,Major_category,Median,ShareWomen
0,ACTUARIAL SCIENCE,Business,62000,0.535714
1,COMPUTER SCIENCE,Computers & Mathematics,53000,0.578766


`OR` example:
```SQL
SELECT Major, Median, Unemployed
FROM recent_grads
WHERE Median >= 10000 OR Unemployed <= 1000 LIMIT 20
```

In [10]:
# Example with OR operator
pd.read_sql_query("\
SELECT Major, Median, Unemployed \
FROM recent_grads \
WHERE Median >= 10000 OR Unemployed <= 1000 LIMIT 20",
                  conn)

,Major,Median,Unemployed
0,PETROLEUM ENGINEERING,110000,37
1,MINING AND MINERAL ENGINEERING,75000,85
2,METALLURGICAL ENGINEERING,73000,16
3,NAVAL ARCHITECTURE AND MARINE ENGINEERING,70000,40
4,CHEMICAL ENGINEERING,65000,1672
5,NUCLEAR ENGINEERING,65000,400
6,ACTUARIAL SCIENCE,62000,308
7,ASTRONOMY AND ASTROPHYSICS,62000,33
8,MECHANICAL ENGINEERING,60000,4650
9,ELECTRICAL ENGINEERING,60000,3895


Example with AND, OR and PARENTHESES:
```SQL
SELECT Major, Major_category, ShareWomen, Unemployment_rate
FROM recent_grads
WHERE (Major_category = 'Engineering') AND (ShareWomen > 0.5 OR Unemployment_rate < 0.051)
```

In [11]:
# Example with AND, OR and PARENTHESES
pd.read_sql_query("\
SELECT Major, Major_category, ShareWomen, Unemployment_rate \
FROM recent_grads \
WHERE (Major_category = 'Engineering') AND (ShareWomen > 0.5 OR Unemployment_rate < 0.051)",
                 conn)

,Major,Major_category,ShareWomen,Unemployment_rate
0,PETROLEUM ENGINEERING,Engineering,0.120564,0.018381
1,METALLURGICAL ENGINEERING,Engineering,0.153037,0.024096
2,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,0.107313,0.050125
3,MATERIALS SCIENCE,Engineering,0.310820,0.023043
4,ENGINEERING MECHANICS PHYSICS AND SCIENCE,Engineering,0.183985,0.006334
5,INDUSTRIAL AND MANUFACTURING ENGINEERING,Engineering,0.343473,0.042876
6,MATERIALS ENGINEERING AND MATERIALS SCIENCE,Engineering,0.292607,0.027789
7,ENVIRONMENTAL ENGINEERING,Engineering,0.558548,0.093589
8,INDUSTRIAL PRODUCTION TECHNOLOGIES,Engineering,0.750473,0.028308
9,ENGINEERING AND INDUSTRIAL MANAGEMENT,Engineering,0.174123,0.033652


## Ordering Results Using ORDER BY

The results of every query we've written so far have been ordered by the `Rank` column. Recall a query from early in the mission, where we wrote a query that returned all of the columns and didn't filter rows on any specific criteria (`SELECT * FROM recent_grads LIMIT 5`):

```SQL
SELECT *
FROM recent_grads
LIMIT 5
```

In [12]:
pd.read_sql_query("\
SELECT * \
FROM recent_grads \
LIMIT 5",
                  conn)

,index,Rank,Major_code,Major,Major_category,Total,Sample_size,Men,Women,ShareWomen,...,Part_time,Full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th,College_jobs,Non_college_jobs,Low_wage_jobs
0,0,1,2419,PETROLEUM ENGINEERING,Engineering,2339,36,2057,282,0.120564,...,270,1207,37,0.018381,110000,95000,125000,1534,364,193
1,1,2,2416,MINING AND MINERAL ENGINEERING,Engineering,756,7,679,77,0.101852,...,170,388,85,0.117241,75000,55000,90000,350,257,50
2,2,3,2415,METALLURGICAL ENGINEERING,Engineering,856,3,725,131,0.153037,...,133,340,16,0.024096,73000,50000,105000,456,176,0
3,3,4,2417,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,1258,16,1123,135,0.107313,...,150,692,40,0.050125,70000,43000,80000,529,102,0
4,4,5,2405,CHEMICAL ENGINEERING,Engineering,32260,289,21239,11021,0.341631,...,5180,16697,1672,0.061098,65000,50000,75000,18314,4440,972


As the questions we want to answer get more complex, we want more control over how the results are ordered. We can specify the order using the [`ORDER BY`](https://sqlite.org/lang_select.html#orderby) clause. For example, we may want to understand which majors that met the criteria in the `WHERE` statement had the lowest unemployment rate. This will return the results in ascending order (increasing) by the `Unemployment_rate` column:

```SQL
SELECT Rank, Major, Major_category, ShareWomen, Unemployment_rate
FROM recent_grads
WHERE (Major_category = 'Engineering') AND (ShareWomen > 0.5 OR Unemployment_rate < 0.051)
ORDER BY Unemployment_rate
```

In [13]:
pd.read_sql_query("\
SELECT Rank, Major, Major_category, ShareWomen, Unemployment_rate \
FROM recent_grads \
WHERE (Major_category = 'Engineering') AND (ShareWomen > 0.5 OR Unemployment_rate < 0.051) \
ORDER BY Unemployment_rate",
                 conn)

,Rank,Major,Major_category,ShareWomen,Unemployment_rate
0,15,ENGINEERING MECHANICS PHYSICS AND SCIENCE,Engineering,0.183985,0.006334
1,1,PETROLEUM ENGINEERING,Engineering,0.120564,0.018381
2,14,MATERIALS SCIENCE,Engineering,0.310820,0.023043
3,3,METALLURGICAL ENGINEERING,Engineering,0.153037,0.024096
4,24,MATERIALS ENGINEERING AND MATERIALS SCIENCE,Engineering,0.292607,0.027789
5,39,INDUSTRIAL PRODUCTION TECHNOLOGIES,Engineering,0.750473,0.028308
6,51,ENGINEERING AND INDUSTRIAL MANAGEMENT,Engineering,0.174123,0.033652
7,17,INDUSTRIAL AND MANUFACTURING ENGINEERING,Engineering,0.343473,0.042876
8,4,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,0.107313,0.050125
9,31,ENVIRONMENTAL ENGINEERING,Engineering,0.558548,0.093589


If we instead want the results ordered by the same column but in descending order, we can add the `DESC` keyword:

```SQL
SELECT Rank, Major, Major_category, ShareWomen, Unemployment_rate
FROM recent_grads
WHERE (Major_category = 'Engineering') AND (ShareWomen > 0.5 OR Unemployment_rate < 0.051)
ORDER BY Unemployment_rate DESC
```

In [14]:
pd.read_sql_query("\
SELECT Rank, Major, Major_category, ShareWomen, Unemployment_rate \
FROM recent_grads \
WHERE (Major_category = 'Engineering') AND (ShareWomen > 0.5 OR Unemployment_rate < 0.051) \
ORDER BY Unemployment_rate DESC",
                 conn)

,Rank,Major,Major_category,ShareWomen,Unemployment_rate
0,31,ENVIRONMENTAL ENGINEERING,Engineering,0.558548,0.093589
1,4,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,0.107313,0.050125
2,17,INDUSTRIAL AND MANUFACTURING ENGINEERING,Engineering,0.343473,0.042876
3,51,ENGINEERING AND INDUSTRIAL MANAGEMENT,Engineering,0.174123,0.033652
4,39,INDUSTRIAL PRODUCTION TECHNOLOGIES,Engineering,0.750473,0.028308
5,24,MATERIALS ENGINEERING AND MATERIALS SCIENCE,Engineering,0.292607,0.027789
6,3,METALLURGICAL ENGINEERING,Engineering,0.153037,0.024096
7,14,MATERIALS SCIENCE,Engineering,0.310820,0.023043
8,1,PETROLEUM ENGINEERING,Engineering,0.120564,0.018381
9,15,ENGINEERING MECHANICS PHYSICS AND SCIENCE,Engineering,0.183985,0.006334


```SQL
SELECT Major, ShareWomen, Unemployment_rate
FROM recent_grads
WHERE (ShareWomen > 0.3) AND (Unemployment_rate < 0.1)
ORDER BY ShareWomen DESC"
```

In [15]:
pd.read_sql_query("\
SELECT Major, ShareWomen, Unemployment_rate \
FROM recent_grads \
WHERE (ShareWomen > 0.3) AND (Unemployment_rate < 0.1) \
ORDER BY ShareWomen DESC",
                  conn)

,Major,ShareWomen,Unemployment_rate
0,EARLY CHILDHOOD EDUCATION,0.967998,0.040105
1,MATHEMATICS AND COMPUTER SCIENCE,0.927807,0.000000
2,ELEMENTARY EDUCATION,0.923745,0.046586
3,ANIMAL SCIENCES,0.910933,0.050862
4,PHYSIOLOGY,0.906677,0.069163
5,MISCELLANEOUS PSYCHOLOGY,0.905590,0.051908
6,HUMAN SERVICES AND COMMUNITY ORGANIZATION,0.904075,0.037819
7,NURSING,0.896019,0.044863
8,GEOSCIENCES,0.881294,0.024374
9,MASS MEDIA,0.877228,0.089837


Let's find out which engineering majors had the highest full time employment rates?

```SQL
SELECT Major_category, Major, Unemployment_rate
FROM recent_grads
WHERE (Major_category = 'Engineering') OR (Major_category = 'Physical Sciences')
ORDER BY Unemployment_rate
```

In [16]:
pd.read_sql_query("\
SELECT Major_category, Major, Unemployment_rate \
FROM recent_grads \
WHERE (Major_category = 'Engineering') OR (Major_category = 'Physical Sciences') \
ORDER BY Unemployment_rate",
                 conn)

,Major_category,Major,Unemployment_rate
0,Engineering,ENGINEERING MECHANICS PHYSICS AND SCIENCE,0.006334
1,Engineering,PETROLEUM ENGINEERING,0.018381
2,Physical Sciences,ASTRONOMY AND ASTROPHYSICS,0.021167
3,Physical Sciences,ATMOSPHERIC SCIENCES AND METEOROLOGY,0.022229
4,Engineering,MATERIALS SCIENCE,0.023043
5,Engineering,METALLURGICAL ENGINEERING,0.024096
6,Physical Sciences,GEOSCIENCES,0.024374
7,Engineering,MATERIALS ENGINEERING AND MATERIALS SCIENCE,0.027789
8,Engineering,INDUSTRIAL PRODUCTION TECHNOLOGIES,0.028308
9,Engineering,ENGINEERING AND INDUSTRIAL MANAGEMENT,0.033652


## Introduction to Summary Statistics with COUNT

Now we'll learn how to calculate summary statistics on subsets of a database table. We'll continue working with data on job outcomes, compiled by FiveThirtyEight.

Let's start with some motivating questions we want to answer:

* How many majors had mostly female students? How many had mostly male students? What proportion of majors had mostly female students?
* Which category of majors had the lowest unemployment rates? Which category of majors had the highest female representation?
* Which majors had the largest spread (difference) between the 25th and 75th percentile starting salaries?

Let's focus on the first set of questions around gender representation:

Instead of returning all of the rows, we want SQLite to count the number of rows and return just that value. While we don't need to change the subset of data we're working with, we do need to change how it's presented to us. To return just the count, we need to use the SQL function [`COUNT()`](https://sqlite.org/lang_aggfunc.html#count):

```SQL
SELECT COUNT(Major)
FROM recent_grads
WHERE ShareWomen > 0.5
```

In [17]:
pd.read_sql_query("\
SELECT COUNT(Major) \
FROM recent_grads \
WHERE ShareWomen > 0.5",
                 conn)

,COUNT(Major)
0,97


Instead of just returning a single value, SQLite returned a table with a column (`COUNT(Major)`) and the count as a row in that column (`97`).

A key idea in SQL is that **everything is a table**. One advantage of this simplification is that it's a common, visual representation that makes SQL approachable for a much wider audience. The disadvantage is that datasets and calculations that aren't well suited for this representation must be converted to be used in a SQL database environment.

## Aggregate Functions

Functions like `COUNT()` are known as [aggregate functions](https://sqlite.org/lang_aggfunc.html). Aggregate functions are applied over columns of values and return a single value. [MIN()](https://sqlite.org/lang_corefunc.html#minoreunc) and [MAX()](https://sqlite.org/lang_corefunc.html#maxoreunc), for example, calculate and return the minimum and maximum values in a column.

We can use these functions to compute the lowest value in the `ShareWomen` column and to know the major.

```SQL
SELECT Major, MIN(ShareWomen)
FROM recent_grads
```

In [18]:
pd.read_sql_query("\
SELECT Major, MIN(ShareWomen) \
FROM recent_grads",
                 conn)

,Major,MIN(ShareWomen)
0,MISCELLANEOUS ENGINEERING TECHNOLOGIES,0.0


If you think about it, `MIN(ShareWomen)` acts a row filter in some way. While the query `SELECT Major FROM recent_grads` returns all of the values in the `Major` column, the query `SELECT Major, MIN(ShareWomen) FROM recent_grads` only returned the `Major` column value corresponding for the row with the minimum value in the `ShareWomen` column.

One thing to note is that while `COUNT()` can be used on any column (because it's just counting the number of values), the other aggregate functions (`MIN()`, `MAX()`, etc) can only be used on numeric columns (since these arithmetic calculations only work with numbers).

Let's write a query that returns the Engineering major with the lowest median salary:

```SQL
SELECT Major, Major_category, MIN(Median)
FROM recent_grads
WHERE Major_category = 'Engineering'
```

In [19]:
pd.read_sql_query("\
SELECT Major, Major_category, MIN(Median) \
FROM recent_grads \
WHERE Major_category = 'Engineering'",
                 conn)

,Major,Major_category,MIN(Median)
0,ARCHITECTURE,Engineering,40000


## Calculating Sums and Averages in SQL

The final two aggregation functions we'll look at are `SUM()` and `AVG()`. Applying the `SUM()` function will add all of the values in a column while `AVG()` will compute the average. Lastly, the `TOTAL()` function also returns the sum as a floating point value (even if the column contains integers). The `TOTAL()` function should be used when working with a column containing floating point values. You can read more [here](https://sqlite.org/lang_aggfunc.html).

This time around, we're going to skip showing sample code since these functions are used the same way as `COUNT()`, `MIN()`, and `MAX()`. This is good practice working with new functions, as SQL contains many functions that you'll end up using down the road that you haven't been taught explicitly.

Let's write a query that computes the sum of the `Total` column:

```SQL
SELECT SUM(Total)
FROM recent_grads
```

In [20]:
pd.read_sql_query("\
SELECT SUM(Total) \
FROM recent_grads",
                 conn)

,SUM(Total)
0,6776015


## Combining Multiple Aggregation Functions

Instead of writing an individual query for each specific question we want to answer, we can actually write queries that answer multiple questions at once. Let's take the following questions:

* What's the lowest median salary?
* What's the highest median salary?
* What's the total number of students?

We can select multiple columns by including their names with commas and we can apply the same principle to combine multiple aggregation functions into a single query. Let's write a query that computes the average of the `Total` column, the minimum of the `Men` column, and the maximum of the `Women` column, in that specific order:

```SQL
SELECT AVG(Total), MIN(Men), MAX(Women)
FROM recent_grads
```

In [21]:
pd.read_sql_query("\
SELECT AVG(Total), MIN(Men), MAX(Women) \
FROM recent_grads",
                  conn)

,AVG(Total),MIN(Men),MAX(Women)
0,39167.716763,119,307087


## Customizing The Results

All of the queries we've written so far have had somewhat unpleasant column names in the results, like AVG(SUM) and MIN(Men). Many companies use SQL environments and tools that can run your query, turn the results into a plot of your choosing, and then create a PDF report containing multiple plots (and some additional explanation from the user). Given that others may interpret and understand the results of your SQL queries, it's helpful to be able to specify custom names for the columns in our results.

We can do just that using the `AS` operator. This is known as an [alias](https://www.tutorialspoint.com/sqlite/sqlite_alias_syntax.htm) and the alias is restricted to just our results table (the table in the database won't be renamed). We can specify an arbitrary phrase as a string using quotation marks or, even better, we can drop AS entirely and just add the name next to the original column:

```SQL
SELECT COUNT(*) as num_students FROM recent_grads
SELECT COUNT(*) as "Total Students" FROM recent_grads
SElECT COUNT(*) "Total Students" FROM recent_grads
```

Lastly, we can reference renamed columns when writing longer queries to make our code more compact:

```SQL
SELECT Major m, Major_category mc, Unemployment_rate ur
FROM recent_grads
WHERE (mc = 'Engineering') AND (ur > 0.04 and ur < 0.08)
ORDER BY ur DESC
```

Let's write a query that returns, in the following order:

* the number of rows as `Number of Students`
* the maximum value of `Unemployment_rate` as `Highest Unemployment Rate`

```SQL
SELECT
COUNT(*) AS "Number of Students",
MAX(Unemployment_rate) AS "Highest Unemployment Rate"
FROM recent_grads
```

In [22]:
pd.read_sql_query('\
SELECT \
COUNT(*) AS "Number of Students", \
MAX(Unemployment_rate) AS "Highest Unemployment Rate" \
FROM recent_grads',
                 conn)

,Number of Students,Highest Unemployment Rate
0,173,0.177226


## Counting Unique Values

We've been working with the Major_category column a decent amount in our queries and it's a column with only few unique values. What if we want to return just the unique values in this column? Or the number of unique values in this column?

We can return all of the unique values in a column using the `DISTINCT` statement:

```SQL
SELECT DISTINCT Major_category 
FROM recent_grads
```

In [23]:
pd.read_sql_query("SELECT DISTINCT Major_category FROM recent_grads", conn)

,Major_category
0,Engineering
1,Business
2,Physical Sciences
3,Law & Public Policy
4,Computers & Mathematics
5,Agriculture & Natural Resources
6,Industrial Arts & Consumer Services
7,Arts
8,Health
9,Social Science


As with the other SQL clauses we've learned, we can use the `DISTINCT` statement with multiple columns to return unique pairings of those columns:

```SQL
SELECT DISTINCT Major, Major_category 
FROM recent_grads 
LIMIT 5;
```

In [24]:
pd.read_sql_query("SELECT DISTINCT Major, Major_category FROM recent_grads LIMIT 5", conn)

,Major,Major_category
0,PETROLEUM ENGINEERING,Engineering
1,MINING AND MINERAL ENGINEERING,Engineering
2,METALLURGICAL ENGINEERING,Engineering
3,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering
4,CHEMICAL ENGINEERING,Engineering


In this case, the `Major_category` column is much more unique (only 16 unique values for `Major_category` compared to 173 for `Major`), so the corresponding value is repeated for every unique value in `Major`.

Lastly, we can count the number of unique values in a column by nesting the `COUNT()` function with the `DISTINCT()` function (note the nesting of parentheses as well):

```SQL
SELECT COUNT(DISTINCT(Major_category)) unique_major_categories
FROM recent_grads
```

In [25]:
pd.read_sql_query("\
SELECT COUNT(DISTINCT(Major_category)) unique_major_categories \
FROM recent_grads",
                  conn)

,unique_major_categories
0,16


Now, let's write a query that returns the number of unique values in the `Major`, `Major_category`, and `Major_code` columns:

```SQL
SELECT 
COUNT(DISTINCT(Major)) unique_majors,
COUNT(DISTINCT(Major_category)) unique_major_categories,
COUNT(DISTINCT(Major_code)) unique_major_codes
FROM recent_grads
```

In [26]:
pd.read_sql_query("\
SELECT COUNT(DISTINCT(Major)) unique_majors, \
COUNT(DISTINCT(Major_category)) unique_major_categories, \
COUNT(DISTINCT(Major_code)) unique_major_codes \
FROM recent_grads",
                 conn)

,unique_majors,unique_major_categories,unique_major_codes
0,173,16,173


## Performing Arithmetic in SQL

Which majors had the largest spread (difference) between the 25th and 75th percentile starting salaries?

To answer this question, we need to be able to perform arithmetic on the columns in a table to compute the difference. SQL supports the standard arithmetic operators: `*`, `+`, `-`, and `/`, and we can use them like any other operator:

```SQL
SELECT P75th - P25th quartile_spread 
FROM recent_grads 
LIMIT 10;
```

In [27]:
pd.read_sql_query("\
SELECT P75th - P25th quartile_spread FROM recent_grads LIMIT 10",
                 conn)

,quartile_spread
0,30000
1,35000
2,55000
3,37000
4,25000
5,52000
6,19000
7,77500
8,22000
9,27000


One thing to note is that multiplying or dividing columns with a floating point value (or a column with floating point values) will result in floating point values:

* Two floats - Returns a float.
    * `SELECT 100.0 / 100.0` returns 1.0.
* A float and an integer - Returns a float
    * `SELECT 100 / 1.0` returns 100.0.
* Two integers - Returns an integer
    * `SELECT 100 / 10` returns 10
    
Now let's write a query that computes the difference between the 25th and 75th percentile of salaries for all majors:

```SQL
SELECT Major, Major_category, P75th - P25th quartile_spread
FROM recent_grads
ORDER BY quartile_spread
LIMIT 20;
```

In [28]:
pd.read_sql_query("\
SELECT Major, Major_category, P75th - P25th quartile_spread \
FROM recent_grads \
ORDER BY quartile_spread \
LIMIT 20",
                 conn)

,Major,Major_category,quartile_spread
0,MILITARY TECHNOLOGIES,Industrial Arts & Consumer Services,0
1,SCHOOL STUDENT COUNSELING,Education,2000
2,LIBRARY SCIENCE,Education,2000
3,COURT REPORTING,Law & Public Policy,4000
4,PHARMACOLOGY,Biology & Life Science,5000
5,EDUCATIONAL ADMINISTRATION AND SUPERVISION,Education,6000
6,COUNSELING PSYCHOLOGY,Psychology & Social Work,6800
7,SPECIAL NEEDS EDUCATION,Education,10000
8,MATHEMATICS TEACHER EDUCATION,Education,10000
9,SOCIAL WORK,Psychology & Social Work,10000


## Group Summary Statistics

In many cases, we want to drill down even more and compute summary statistics per group. We'll explore how to calculate more granular summary statistics using groups.

We'll drill down and compute summary statistics by group to answer questions like:

* What's the share of women in each major category?
* Which major categories have the greatest numbers of employed graduates?
* What percentage of people in each major category end up in low-wage jobs?

The `GROUP BY` SQL statement allows us to compute summary statistics by "group," or unique value. When we use this statement, SQL creates a group for each unique value in a column or set of columns (the same values we get when we use the `DISTINCT` statement), and then does the calculations for them. To illustrate, we can find the total number of people employed in each major category with the following query:

```SQL
SELECT Major_category, SUM(Employed)
FROM recent_grads
GROUP BY Major_category
```

In [29]:
pd.read_sql_query("\
SELECT Major_category, SUM(Employed) \
FROM recent_grads \
GROUP BY Major_category",
                 conn)

,Major_category,SUM(Employed)
0,Agriculture & Natural Resources,66943
1,Arts,288114
2,Biology & Life Science,302797
3,Business,1088742
4,Communications & Journalism,330660
5,Computers & Mathematics,237894
6,Education,479839
7,Engineering,420372
8,Health,372147
9,Humanities & Liberal Arts,544118


This gives us the total number of employed graduates for each major category. The `GROUP BY` statement splits the `Major_category` column into groups (with one group for each unique major category), then calculates the sum for each group.

If a column is selected, the SQL engine will use the **last** value for that column in the group. If an aggregation function is selected, the SQL engine will compute the value for that aggregation function across the group:

```SQL
SELECT Employed, Major_category, SUM(Employed)
FROM recent_grads
GROUP BY Major_category
```

In [30]:
pd.read_sql_query("\
SELECT Employed, Major_category, SUM(Employed) \
FROM recent_grads \
GROUP BY Major_category",
                 conn)

,Employed,Major_category,SUM(Employed)
0,3149,Agriculture & Natural Resources,66943
1,2914,Arts,288114
2,1144,Biology & Life Science,302797
3,2912,Business,1088742
4,179633,Communications & Journalism,330660
5,102087,Computers & Mathematics,237894
6,730,Education,479839
7,1976,Engineering,420372
8,180903,Health,372147
9,2787,Humanities & Liberal Arts,544118


Now, let's have some practice. We'll find the percentage of graduates who are employed for each major category:

```SQL
SELECT Major_category, AVG(Employed) / AVG(Total) share_employed
FROM recent_grads
GROUP BY Major_category
```

In [31]:
pd.read_sql_query("\
SELECT Major_category, AVG(Employed) / AVG(Total) share_employed \
FROM recent_grads \
GROUP BY Major_category",
                 conn)

,Major_category,share_employed
0,Agriculture & Natural Resources,0.836986
1,Arts,0.806748
2,Biology & Life Science,0.667157
3,Business,0.835966
4,Communications & Journalism,0.842229
5,Computers & Mathematics,0.795611
6,Education,0.858190
7,Engineering,0.781967
8,Health,0.803374
9,Humanities & Liberal Arts,0.762638


## Querying Virtual Columns With the HAVING Statement

Sometimes we want to select a subset of rows after performing a `GROUP BY` query. Previously, for instance, we may have wanted to select only those rows where share_employed is greater than `.8`. We can't use the `WHERE` clause to do this because `share_employed` isn't a column in `recent_grads`; it's actually a virtual column generated by the `GROUP BY` statement.

When we want to filter on a column generated by a `GROUP BY` query, we can use the `HAVING` statement. Here's an example:

```SQL
SELECT Major_category, AVG(Employed) / AVG(Total) AS share_employed 
FROM recent_grads 
GROUP BY Major_category 
HAVING share_employed > .8;
```

Note that we used the same column name in the `HAVING` statement that we originally specified with the `AS` statement. SQL allows us to use custom column names in subsequent statements, including `HAVING` and `WHERE`. The statement below will result in the following output:

In [32]:
pd.read_sql_query("\
SELECT Major_category, AVG(Employed) / AVG(Total) AS share_employed \
FROM recent_grads \
GROUP BY Major_category \
HAVING share_employed > .8;",
                 conn)

,Major_category,share_employed
0,Agriculture & Natural Resources,0.836986
1,Arts,0.806748
2,Business,0.835966
3,Communications & Journalism,0.842229
4,Education,0.858190
5,Health,0.803374
6,Industrial Arts & Consumer Services,0.822670
7,Law & Public Policy,0.808399


Note that the results only include categories where share_employed is greater than `.8`. That's because the `HAVING` statement filters out the other rows. Let's find all of the major categories where the share of graduates with low-wage jobs is greater than `.1`:

```SQL
SELECT Major_category, AVG(Low_wage_jobs) / AVG(Total) AS share_low_wage
FROM recent_grads
GROUP BY Major_category
HAVING share_low_wage > 0.1
```

In [33]:
pd.read_sql_query("\
SELECT Major_category, AVG(Low_wage_jobs) / AVG(Total) AS share_low_wage \
FROM recent_grads \
GROUP BY Major_category \
HAVING share_low_wage > 0.1",
                 conn)

,Major_category,share_low_wage
0,Arts,0.168331
1,Communications & Journalism,0.126324
2,Humanities & Liberal Arts,0.132087
3,Industrial Arts & Consumer Services,0.115713
4,Law & Public Policy,0.115685
5,Psychology & Social Work,0.116934
6,Social Science,0.102233


## Rounding Results With the ROUND() Function

The percentages in our results were very long and hard to read (e.g., 0.16833085991095678). We can use the SQL `ROUND` function in our query to round them.

```SQL
SELECT Major_category, ROUND(ShareWomen, 2) AS rounded_share_women 
FROM recent_grads
LIMIT 10;
```

The query will round the `ShareWomen` column to two decimal places. Here's a truncated view of the results:

In [34]:
pd.read_sql_query("\
SELECT Major_category, ROUND(ShareWomen, 2) AS rounded_share_women \
FROM recent_grads \
LIMIT 10",
                 conn)

,Major_category,rounded_share_women
0,Engineering,0.12
1,Engineering,0.10
2,Engineering,0.15
3,Engineering,0.11
4,Engineering,0.34
5,Engineering,0.14
6,Business,0.54
7,Physical Sciences,0.44
8,Engineering,0.14
9,Engineering,0.44


Now let's use all of this to find share of employed graduates:

```SQL
SELECT Major_category, ROUND(AVG(Employed) / AVG(Total), 3) AS share_employed
FROM recent_grads
GROUP BY Major_category
HAVING share_employed > .8;
```

In [35]:
pd.read_sql_query("\
SELECT Major_category, ROUND(AVG(Employed) / AVG(Total), 3) AS share_employed \
FROM recent_grads \
GROUP BY Major_category \
HAVING share_employed > .8;",
                 conn)

,Major_category,share_employed
0,Agriculture & Natural Resources,0.837
1,Arts,0.807
2,Business,0.836
3,Communications & Journalism,0.842
4,Education,0.858
5,Health,0.803
6,Industrial Arts & Consumer Services,0.823
7,Law & Public Policy,0.808


## Casting

We used SQL arithmetic to divide float columns. This resulted in float values that we could round using the `ROUND()` function. We can use the [`PRAGMA TABLE_INFO()`](https://sqlite.org/pragma.html#pragma_table_info) statement by itself to return the type, along with some other information, for each column:

```SQL
PRAGMA TABLE_INFO(recent_grads)
```

In [36]:
pd.read_sql_query("PRAGMA TABLE_INFO(recent_grads)",
                 conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Rank,INTEGER,0,None,0
2,2,Major_code,INTEGER,0,None,0
3,3,Major,TEXT,0,None,0
4,4,Major_category,TEXT,0,None,0
5,5,Total,INTEGER,0,None,0
6,6,Sample_size,INTEGER,0,None,0
7,7,Men,INTEGER,0,None,0
8,8,Women,INTEGER,0,None,0
9,9,ShareWomen,REAL,0,None,0


If we try to divide 2 integer columns (`Women` and `Total`), SQL will round down and return integer values. We need to instead use the `CAST()` function to the Float type:

```SQL
SELECT Major_category,
CAST(SUM(Women) AS Float) / CAST(SUM(Total) AS Float) AS SW
FROM recent_grads
GROUP BY Major_category
ORDER BY SW
```

In [37]:
pd.read_sql_query("\
SELECT Major_category, \
CAST(SUM(Women) AS Float) / CAST(SUM(Total) AS Float) AS SW \
FROM recent_grads \
GROUP BY Major_category \
ORDER BY SW",
                 conn)

,Major_category,SW
0,Law & Public Policy,0.030585
1,Business,0.084743
2,Industrial Arts & Consumer Services,0.160249
3,Computers & Mathematics,0.209356
4,Engineering,0.219596
5,Communications & Journalism,0.250325
6,Arts,0.393327
7,Humanities & Liberal Arts,0.490051
8,Health,0.673588
9,Interdisciplinary,0.800911


## Writing More Complex Queries

The SQL operations we've learned so far enable us to answer questions with only one source of uncertainty. Many times, we want to answer questions that have 2 or more levels of unknowns. For example:

* Which rows are above the average for the ShareWomen column?

Using the SQL techniques we've learned so far, there's no way to write a query that answers this question. As of right now, we only know aggregate functions such as `AVG()` is valid in the `SELECT` clause; however, they can be used in other clauses such as the `GROUP BY` and `HAVING` clauses. For example, The following query:

```SQL
SELECT * FROM recent_grads
WHERE ShareWomen > AVG(ShareWomen)
```

will return an error:

```python
(sqlite3.OperationalError) misuse of aggregate function AVG() [SQL: 'SELECT * FROM recent_grads WHERE ShareWomen > AVG(ShareWomen)']
```

We need to instead learn how to break up a question we want to answer into a series of queries that can be combined.

To determine which majors are above the average for the `ShareWomen` column, we need to:

* first determine the average value for the `ShareWomen` column
* then select and filter the rows that are greater than the average value

If we had to do this using Python and pandas, we would compute and store the average value in `ShareWomen` as a variable and then use the variable in a table filter. While variables dominate how we express logic in object-oriented programming languages like Python and Java, SQL doesn't have support for variables. The designers of SQL, a [declarative programming language](https://en.wikipedia.org/wiki/Declarative_programming), want it's users to focus on expressing computations over explicitly defining, setting, and juggling variables.

What would the query look like if we already knew the average value for the `ShareWomen` column?

```SQL
SELECT Major, ShareWomen FROM recent_grads
WHERE ShareWomen > 0.5225502029537575
```



Now, how do we make the computed average value, 0.5225502029537575, dynamic?

Let's introduce the SQL way to solve this problem -- **subqueries**. A subquery is a query nested within another query. Here's a template for a SQL statement where the subquery resides in the `WHERE` clause:

```SQL
SELECT Major, ShareWomen FROM recent_grads
WHERE ShareWomen > (subquery that returns the average value for ShareWomen)
```

The subquery is run first and returns the average value for the `ShareWomen` column (which happens to be 0.5225502029537575). Based on the result of the subquery, SQL will replace the subquery with this value dynamically. Note that SQL will ignore the column name (`AVG(ShareWomen)`) and is smart enough to just use the actual row value.

The query that replaces the placeholder `subquery` needs to be a full query (contain `SELECT` and `FROM` clauses, etc), that works even if it's run separately. In addition, the inner query should only return a table with a single row and column because of where it fits in the outer query (`... WHERE > ?`).

A subquery must always be contained within parentheses `()`.

```SQL
SELECT Major, Unemployment_rate
FROM recent_grads
WHERE Unemployment_rate < (SELECT AVG(Unemployment_rate) FROM recent_grads)
ORDER BY Unemployment_rate
```

In [38]:
pd.read_sql_query("\
SELECT Major, Unemployment_rate \
FROM recent_grads \
WHERE Unemployment_rate < (SELECT AVG(Unemployment_rate) FROM recent_grads) \
ORDER BY Unemployment_rate",
                 conn)

,Major,Unemployment_rate
0,MATHEMATICS AND COMPUTER SCIENCE,0.000000
1,BOTANY,0.000000
2,SOIL SCIENCE,0.000000
3,EDUCATIONAL ADMINISTRATION AND SUPERVISION,0.000000
4,ENGINEERING MECHANICS PHYSICS AND SCIENCE,0.006334
5,COURT REPORTING,0.011690
6,MATHEMATICS TEACHER EDUCATION,0.016203
7,PETROLEUM ENGINEERING,0.018381
8,GENERAL AGRICULTURE,0.019642
9,ASTRONOMY AND ASTROPHYSICS,0.021167


## Subquery in SELECT

What if we wanted to understand the proportion of majors are above the average for a given column? We'd need to divide the number of rows that met the filter criteria with the total number of rows in the table.

Using the `COUNT()` aggregate function, we can return the number of rows the results set contains:

```SQL
SELECT COUNT(*) 
FROM recent_grads
WHERE ShareWomen > (SELECT AVG(ShareWomen) FROM recent_grads)
```

In [39]:
pd.read_sql_query("\
SELECT COUNT(*) FROM recent_grads \
WHERE ShareWomen > (SELECT AVG(ShareWomen) FROM recent_grads)",
                 conn)

,COUNT(*)
0,91


To return the proportion, we need to divide this value with the total number of rows in `recent_grads`. The challenge, however, is that the we don't know the total number of rows (or want to be reliant on an out of date calculation anyway that we could potentially hard code).

To dynamically calculate the number of total rows in `recent_grads` and be able to use it in another SQL statement, we can use a subquery in the `SELECT` clause:

```SQL
SELECT 
COUNT(*), 
(SELECT COUNT(*) FROM recent_grads) 
FROM recent_grads
WHERE ShareWomen > (SELECT AVG(ShareWomen) FROM recent_grads)
```

In [40]:
pd.read_sql_query("\
SELECT COUNT(*), (SELECT COUNT(*) FROM recent_grads) FROM recent_grads \
WHERE ShareWomen > (SELECT AVG(ShareWomen) FROM recent_grads)",
                 conn)

,COUNT(*),(SELECT COUNT(*) FROM recent_grads)
0,91,173


Now, let's find the proportion

```SQL
SELECT 
CAST(COUNT(*) AS Float) / CAST((SELECT COUNT(*) FROM recent_grads) AS Float)
AS proportion_abv_avg 
FROM recent_grads
WHERE ShareWomen > (SELECT AVG(ShareWomen) FROM recent_grads)
```

In [41]:
pd.read_sql_query("\
SELECT CAST(COUNT(*) AS Float) / \
CAST((SELECT COUNT(*) FROM recent_grads) AS Float) \
AS proportion_abv_avg FROM recent_grads \
WHERE ShareWomen > (SELECT AVG(ShareWomen) FROM recent_grads)",
                 conn)

,proportion_abv_avg
0,0.526012


## Returning Multiple Results In Subqueries

So far, the subqueries we've used have computed an aggregate value of some kind and returned that value to the outer query to use for filtering. This is because we only worked with the `<` and `>` operators, which, by definition, expect a single value to compare against in a filter.

SQLite understands the following binary operators, in order from highest to lowest precedence: 

``` SQL
||
* / % 
+ - 
<< >> & | 
< <= > >= 
= == != <> IS IS NOT IN LIKE GLOB MATCH REGEXP 
AND 
OR
```

And supported unary prefix operators are these:

```SQL
- + ~ NOT
```

Using the `IN` operator, we can specify a list of values that we want to match against in the `WHERE` clause. All rows that match exactly will be returned. The following query returns the rows where `Major_category` equals either `Business` or `Engineering`:

```SQL
SELECT Major, Major_category 
FROM recent_grads
WHERE Major_category IN ('Business', 'Engineering')
LIMIT 7
```

In [42]:
pd.read_sql_query("\
SELECT Major, Major_category FROM recent_grads \
WHERE Major_category IN ('Business', 'Engineering') \
LIMIT 7",
                 conn)

,Major,Major_category
0,PETROLEUM ENGINEERING,Engineering
1,MINING AND MINERAL ENGINEERING,Engineering
2,METALLURGICAL ENGINEERING,Engineering
3,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering
4,CHEMICAL ENGINEERING,Engineering
5,NUCLEAR ENGINEERING,Engineering
6,ACTUARIAL SCIENCE,Business


Opportunities like this, where we've hard coded values, are usually good candidates for converting to a subquery. Instead of returning the rows where `Major_category` equals one of 2 specific values, we can write a subquery that returns the `Major_category` with the 5 highest group level sums for the `Total` column:

```SQL
SELECT Major_category FROM recent_grads
GROUP BY Major_category
ORDER BY SUM(Total) DESC
LIMIT 5;
```

In [43]:
pd.read_sql_query("\
SELECT Major_category FROM recent_grads \
GROUP BY Major_category \
ORDER BY SUM(Total) DESC \
LIMIT 5",
                 conn)

,Major_category
0,Business
1,Humanities & Liberal Arts
2,Education
3,Engineering
4,Social Science


Now let's write the whole query:

```SQL
SELECT Major, Major_category
FROM recent_grads
WHERE Major_category IN (
    SELECT Major_category 
    FROM recent_grads
    GROUP BY Major_category
    ORDER BY SUM(Total) DESC
    LIMIT 5);
```

In [44]:
pd.read_sql_query("\
SELECT Major, Major_category \
FROM recent_grads \
WHERE Major_category IN (SELECT Major_category FROM recent_grads \
GROUP BY Major_category \
ORDER BY SUM(Total) DESC \
LIMIT 5)",
                 conn)

,Major,Major_category
0,PETROLEUM ENGINEERING,Engineering
1,MINING AND MINERAL ENGINEERING,Engineering
2,METALLURGICAL ENGINEERING,Engineering
3,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering
4,CHEMICAL ENGINEERING,Engineering
5,NUCLEAR ENGINEERING,Engineering
6,ACTUARIAL SCIENCE,Business
7,MECHANICAL ENGINEERING,Engineering
8,ELECTRICAL ENGINEERING,Engineering
9,COMPUTER ENGINEERING,Engineering


## Building Complex Subqueries

We can actually nest subqueries within subqueries many times, but this makes our SQL code more complex and harder to debug. Later we'll explore other techniques of composing SQL statements that make nested logic easier.

When you have a SQL statement you want to write that will end up using many subqueries, it can be overwhelming at first to know how to start. In general, you want to start with the inner queries first and work your way outwards. Let's say we're interested in understanding the ratio of the `Sample_size` column to the `Total` column.

Specifically, let's say we're interested in:

* computing this ratio for every major
* understanding which majors are above the average for this ratio
* understanding how many majors are above the average for this ratio

We'll start by writing a query that computes the ratio for every major and then the average of all of these ratios:

```SQL
SELECT AVG(CAST(Sample_size AS Float) / CAST(Total AS Float))
AS avg_ratio
FROM recent_grads
```

In [45]:
pd.read_sql_query("\
SELECT AVG(CAST(Sample_size AS Float) / CAST(Total AS Float)) \
AS avg_ratio \
FROM recent_grads",
                  conn)

,avg_ratio
0,0.009086


Now that we have a subquery that calculates the average ratio (of `Sample_size` to `Total`), we can return the rows that exceed this average.

```SQL
SELECT 
Major, 
Major_category,
CAST(Sample_size AS Float) / CAST(Total AS Float) AS ratio
FROM recent_grads
WHERE ratio > (SELECT AVG(CAST(Sample_size AS Float) / CAST(Total AS Float))
FROM recent_grads)
```

In [46]:
pd.read_sql_query("\
SELECT Major, Major_category, \
CAST(Sample_size AS Float) / CAST(Total AS Float) AS ratio \
FROM recent_grads \
WHERE ratio > (SELECT AVG(CAST(Sample_size AS Float) / CAST(Total AS Float)) \
FROM recent_grads)",
                 conn)

,Major,Major_category,ratio
0,PETROLEUM ENGINEERING,Engineering,0.015391
1,MINING AND MINERAL ENGINEERING,Engineering,0.009259
2,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,0.012719
3,ACTUARIAL SCIENCE,Business,0.013503
4,MECHANICAL ENGINEERING,Engineering,0.011280
5,COMPUTER ENGINEERING,Engineering,0.009605
6,AEROSPACE ENGINEERING,Engineering,0.009762
7,INDUSTRIAL AND MANUFACTURING ENGINEERING,Engineering,0.009648
8,ARCHITECTURAL ENGINEERING,Engineering,0.009204
9,COURT REPORTING,Law & Public Policy,0.012195


## Back to Querying SQLite from Python

Before we can execute a query, we need to express our SQL query as a string. While we use the Connection class to represent the database we're working with, we use the [Cursor class](https://docs.python.org/3/library/sqlite3.html#cursor-objects) to:

* Run a query against the database
* Parse the results from the database
* Convert the results to native Python objects
* Store the results within the Cursor instance as a local variable

After running a query and converting the results to a list of **tuples**, the Cursor instance stores the list as a local variable. Before diving into the syntax of querying the database, let's learn more about tuples.

We need to use the Connection instance method `cursor()` to return a Cursor instance corresponding to the database we want to query.

```python
cursor = conn.cursor()
```

In the following code block, we:

* Write a basic `SELECT` query that will return all of the values from the `recent_grads` table, and store this query as a string named `query`
* Use the Cursor method `execute()` to run the query against our database
* Return the full results set and store it as `results`
* Print the first three tuples in the list `results`

```python
# SQL Query as a string
query = "SELECT * FROM recent_grads;"
# Execute the query, convert the results to tuples, and store as a local variable
cursor.execute(query)
# Fetch the full results set as a list of tuples
results = cursor.fetchall()
# Display the first three results
print(results[0:3])
```

Let's have some practice:

In [47]:
import sqlite3
conn = sqlite3.connect("jobs.db")
cursor = conn.cursor()

query = "SELECT Major FROM recent_grads;"
cursor.execute(query)
majors = cursor.fetchall()
print(majors[0:2])

[('PETROLEUM ENGINEERING',), ('MINING AND MINERAL ENGINEERING',)]


So far, we've been running queries by creating a Cursor instance, and then calling the `execute` method on the instance. The SQLite library actually allows us to skip creating a Cursor altogether by using the [`execute` method](https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.execute) within the Connection object itself. SQLite will create a Cursor instance for us under the hood and our query run against the database, but this shortcut allows us to skip a step. Here's what the code looks like:

In [48]:
conn = sqlite3.connect("jobs.db")
query = "SELECT Major FROM recent_grads;"
print(conn.execute(query).fetchall()[0:2])

[('PETROLEUM ENGINEERING',), ('MINING AND MINERAL ENGINEERING',)]


Notice that we didn't explicitly create a separate Cursor instance ourselves in this code example.

Now let's learn how to fetch a specific number of results after we run a query.

To make it easier to work with large results sets, the Cursor class allows us to control the number of results we want to retrieve at any given time. To return a single result (as a tuple), we use the Cursor method [`fetchone()`](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.fetchone). To return `n` results, we use the Cursor method [`fetchmany()`](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.fetchmany).

Each Cursor instance contains an internal counter that updates every time we retrieve results. When we call the `fetchone()` method, the Cursor instance will return a single result, and then increment its internal counter by 1. This means that if we call `fetchone()` again, the Cursor instance will actually return the second tuple in the results set (and increment by 1 again).

The `fetchmany()` method takes in an integer (`n`) and returns the corresponding results, starting from the current position. It then increments the Cursor instance's counter by `n`. In the following code, we return the first two results using the `fetchone()` method, then the next five results using the `fetchmany()` method.

```python
first_result = cursor.fetchone()
second_result = cursor.fetchone()
next_five_results = cursor.fetchmany(5)
```

Now let's write and run a query that returns the `Major` and `Major_category` columns from `recent_grads`. Then, fetch the first five results and store them as `five_results`.

In [49]:
import sqlite3
conn = sqlite3.connect("jobs.db")
cursor = conn.cursor()

query = "SELECT Major, Major_category FROM recent_grads;"
cursor.execute(query)
five_results = cursor.fetchmany(5)
print(five_results)

[('PETROLEUM ENGINEERING', 'Engineering'), ('MINING AND MINERAL ENGINEERING', 'Engineering'), ('METALLURGICAL ENGINEERING', 'Engineering'), ('NAVAL ARCHITECTURE AND MARINE ENGINEERING', 'Engineering'), ('CHEMICAL ENGINEERING', 'Engineering')]


Because SQLite restricts access to the database file when we're connected to a database, we need to close the connection when we're done working with it. Closing the connection allows other processes to access the database, which is important when you're in a production environment and working with other team members.

To close a connection to a database, use the Connection instance method `close()`. When we're working with multiple databases and multiple Connection instances, we want to make sure we call the `close()` method on the correct instance.

```python
conn.close()
```

Here's an example of a typical workflow code:

```python
import sqlite3
conn = sqlite3.connect("jobs2.db")
cursor = conn.cursor()

query = "SELECT Major FROM recent_grads ORDER BY Major DESC"
cursor.execute(query)
reverse_alphabetical = cursor.fetchall()

conn.close()
```

In [50]:
conn.close()

# Intermediate SQL: Table Joins And Relations

---

## Introducing Joins

We worked exclusively with data that existed in a single table. In the real world, it's much more common for databases to have data in more than one table. If we want to be able to work with that data, we'll have to combine multiple tables within a query. The way we do this in SQL is using **joins**.

We'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth in the `facts` table.

In addition to the `facts` table we've added a new table, called `cities` which contains information on [major urban areas](https://www.cia.gov/library/publications/the-world-factbook/docs/notesanddefs.html?fieldkey=2219&term=Major%20urban%20areas%20-%20population) from countries in the Factbook (for the rest of this mission, we'll use the word 'cities' to mean the same as 'major urban areas'). Let's take a look at the first few rows of these tables.

In [51]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("factbook.db")

In [52]:
facts_head_query = "SELECT * FROM facts LIMIT 5"
cities_head_query = "SELECT * FROM cities LIMIT 5"

```SQL
SELECT * 
FROM facts 
LIMIT 5;
```

In [53]:
pd.read_sql_query(facts_head_query, conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
1,2,al,Albania,28748,27398,1350,3029278,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
3,4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The Factbook contains demographic information like:

* `population` - The population as of 2015
* `population_growth` - The annual population growth rate, as a percentage
* `area` - The total land and water area

```SQL
SELECT * 
FROM cities 
LIMIT 5;
```

In [54]:
pd.read_sql_query(cities_head_query, conn)

,id,name,population,capital,facts_id
0,1,Oranjestad,37000,1,216
1,2,Saint John'S,27000,1,6
2,3,Abu Dhabi,942000,1,184
3,4,Dubai,1978000,0,184
4,5,Sharjah,983000,0,184


A description of what each column of the table represents:


* `id` - A unique ID for each city
* `name` - The name of the city
* `population` - The population of the city
* `capital` - Whether the city is a capital city: 1 if it is, 0 if it isn't
* `facts_id` - The ID of the country, from the facts table

The last column (`facts_id`) is of particular interest to us, as it is a column of data that also exists in our original `facts` table. This link between tables is important as it's used to combine the data in our queries.

The most common way to join data using SQL is using an inner join. The syntax for an inner join is:

```SQL
SELECT [column_names] FROM [table_name_one]
INNER JOIN [table_name_two] ON [join_constraint];
```

The inner join clause is made up of two parts:

* `INNER JOIN`, which tells the SQL engine the name of the table you wish to join in your query, and that you wish to use an inner join
* `ON`, which tells the SQL engine what columns to use to join the two tables

Joins are usually used in a query after the `FROM` clause. Let's look at a basic inner join where we combine the data from both of our tables.

```SQL
SELECT * FROM facts
INNER JOIN cities ON cities.facts_id = facts.id
LIMIT 5;
```

Let's look at the line of the query with the join in it:

* `INNER JOIN` cities - This tells the SQL engine that we wish to join the cities table to our query using an inner join
* `ON cities.facts_id = facts.id` - This tells the SQL engine which columns to use when joining the data, following the syntax `table_name.column_name`

You might presume that `SELECT * FROM facts` will mean that the query returns only columns from the `facts` table, however the `*` wildcard when used with a join will give you all columns from both tables. Here is the result of this query:

```SQL
SELECT * FROM facts
INNER JOIN cities ON cities.facts_id = facts.id
LIMIT 5;
```

In [55]:
join_head_query = "\
SELECT * FROM facts \
INNER JOIN cities ON cities.facts_id = facts.id \
LIMIT 5;"

pd.read_sql_query(join_head_query, conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,id,name,population,capital,facts_id
0,216,aa,Aruba,180,180,0,112162,1.33,12.56,8.18,8.92,1,Oranjestad,37000,1,216
1,6,ac,Antigua and Barbuda,442,442,0,92436,1.24,15.85,5.69,2.21,2,Saint John'S,27000,1,6
2,184,ae,United Arab Emirates,83600,83600,0,5779760,2.58,15.43,1.97,12.36,3,Abu Dhabi,942000,1,184
3,184,ae,United Arab Emirates,83600,83600,0,5779760,2.58,15.43,1.97,12.36,4,Dubai,1978000,0,184
4,184,ae,United Arab Emirates,83600,83600,0,5779760,2.58,15.43,1.97,12.36,5,Sharjah,983000,0,184


This query gives us all columns from both tables and every row where there is a match between the `id` column from `facts` and the `facts_id` from `cities`, limited to the first 5 rows.

Here's a basic cheat sheet on SQL joins:

<img src="https://i.imgur.com/PxKTxj8.png" alt="SQL-joins" width="600"/>

## Understanding Inner Joins

We've now joined the two tables to give us extra information about each row in `cities`. Let's take a closer look at how this inner join works.

Our inner join **will include**:

* Rows from the `cities` table that have a `cities.facts_id` that matches a `facts.id` from facts

Our inner join **will not include*:

* Rows from the `cities` table that have a `cities.facts_id` that doesn't match any `facts.id` from `facts`
* Rows from the `facts` table that have a `facts.id` that doesn't match any `cities.facts_id` from `cities`



We can also create aliases for table names, which makes queries with joins easier to both read and write. Instead of:

```SQL
SELECT * FROM facts
INNER JOIN cities ON cities.facts_id = facts.id
```

We can write:

```SQL
SELECT * FROM facts AS f
INNER JOIN cities AS c ON c.facts_id = f.id
```

Just like with column names, using `AS` is optional. We can get the same result by writing:

```SQL
SELECT * FROM facts f
INNER JOIN cities c ON c.facts_id = f.id
```

We can also combine aliases with wildcards - for instance, using the aliases created above, `c.*` would give us all columns from the table cities.

While our query included both columns from the `ON` clause, we don't need to use either column from our `ON` clause in our final list of columns. This is useful as it means we can show only the information we're interested in, rather than having to include the two join columns every time.

Let's practice on it a little bit:

```SQL
SELECT cities.*, facts.name country_name FROM facts
INNER JOIN cities ON cities.facts_id = facts.id
LIMIT 5;
```

Or we could use this:
```SQL
SELECT c.*, f.name country_name FROM facts f
INNER JOIN cities c ON c.facts_id = f.id
LIMIT 5;
```

In [56]:
join_head_query = "\
SELECT cities.*, facts.name country_name FROM facts \
INNER JOIN cities ON cities.facts_id = facts.id \
LIMIT 5;"

# or we could use this:
#join_head_query = "SELECT c.*, f.name country_name FROM facts f \
#INNER JOIN cities c ON c.facts_id = f.id \
#LIMIT 5;"

pd.read_sql_query(join_head_query, conn)

,id,name,population,capital,facts_id,country_name
0,1,Oranjestad,37000,1,216,Aruba
1,2,Saint John'S,27000,1,6,Antigua and Barbuda
2,3,Abu Dhabi,942000,1,184,United Arab Emirates
3,4,Dubai,1978000,0,184,United Arab Emirates
4,5,Sharjah,983000,0,184,United Arab Emirates


Let's write a query to get a table with countries, names of the capital cities and the facts about these capital cities:

```SQL
SELECT facts.name AS country, cities.name AS capital_city
FROM cities
INNER JOIN facts ON facts.id = cities.facts_id
WHERE cities.capital = 1
LIMIT 5;
```

In [57]:
capital_query = "\
SELECT facts.name AS country, cities.name AS capital_city \
FROM cities \
INNER JOIN facts ON facts.id = cities.facts_id \
WHERE cities.capital = 1 \
LIMIT 5;"

pd.read_sql_query(capital_query, conn)

,country,capital_city
0,Aruba,Oranjestad
1,Antigua and Barbuda,Saint John'S
2,United Arab Emirates,Abu Dhabi
3,Afghanistan,Kabul
4,Algeria,Algiers


## Left Joins

As we mentioned earlier, an inner join will not include any rows where there is not a mutual match from both tables. This means there could be information we are not seeing in our query where rows don't match.

We can use some SQL queries to explore this:

```SQL
SELECT COUNT(DISTINCT(id)) 
FROM facts;
```

In [58]:
pd.read_sql_query("\
SELECT COUNT(DISTINCT(id)) FROM facts;",
                 conn)

,COUNT(DISTINCT(id))
0,261


```SQL
SELECT COUNT(DISTINCT(facts_id)) 
FROM cities;
```

In [59]:
pd.read_sql_query("SELECT COUNT(DISTINCT(facts_id)) FROM cities;",
                 conn)

,COUNT(DISTINCT(facts_id))
0,210


By running these two queries, we can see that there are some countries in the `facts` table that don't have corresponding cities in the `cities` table, which indicates we may have some incomplete data.

A left join includes all the rows that an inner join will select, plus any rows from the first (or left) table that don't have a match in the second table.

```SQL
SELECT * 
FROM facts
LEFT JOIN cities ON cities.facts_id = facts.id;
```

In [60]:
pd.read_sql_query("\
SELECT * FROM facts \
LEFT JOIN cities ON cities.facts_id = facts.id",
                 conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,id,name,population,capital,facts_id
0,1,af,Afghanistan,652230.0,652230.0,0.0,3.256434e+07,2.32,38.57,13.89,1.51,6.0,Kabul,3097000.0,1.0,1
1,2,al,Albania,28748.0,27398.0,1350.0,3.029278e+06,0.30,12.92,6.58,3.30,10.0,Tirana,419000.0,1.0,2
2,3,ag,Algeria,2381741.0,2381741.0,0.0,3.954217e+07,1.84,23.67,4.31,0.92,7.0,Algiers,2916000.0,1.0,3
3,3,ag,Algeria,2381741.0,2381741.0,0.0,3.954217e+07,1.84,23.67,4.31,0.92,8.0,Oran,783000.0,0.0,3
4,4,an,Andorra,468.0,468.0,0.0,8.558000e+04,0.12,8.13,6.96,0.00,12.0,Andorra La Vella,23000.0,1.0,4
5,5,ao,Angola,1246700.0,1246700.0,0.0,1.962535e+07,2.78,38.78,11.49,0.46,13.0,Luanda,5068000.0,1.0,5
6,5,ao,Angola,1246700.0,1246700.0,0.0,1.962535e+07,2.78,38.78,11.49,0.46,14.0,Huambo,1098000.0,0.0,5
7,6,ac,Antigua and Barbuda,442.0,442.0,0.0,9.243600e+04,1.24,15.85,5.69,2.21,2.0,Saint John'S,27000.0,1.0,6
8,7,ar,Argentina,2780400.0,2736690.0,43710.0,4.343189e+07,0.93,16.64,7.33,0.00,16.0,Buenos Aires,13528000.0,1.0,7
9,7,ar,Argentina,2780400.0,2736690.0,43710.0,4.343189e+07,0.93,16.64,7.33,0.00,17.0,Cordoba,1556000.0,0.0,7


For the rows where `facts.id` doesn't match any values in `cities.facts_id` (`237`, `238`, `240`, and `244`), the rows are still included in the results. When this happens, all of the columns from the `cities` table are populated with null values. We can use these null values to filter our results to just the countries that don't exist in `cities` with a `WHERE` clause. When making a comparison to null in SQL, we use the `IS` keyword, rather than the `=` sign.

Let's write a query that returns the countries that don't exist in `cities`:

```SQL
SELECT facts.name AS country, cities.population
FROM facts
LEFT JOIN cities ON cities.facts_id = facts.id
WHERE cities.facts_id IS NULL;
```

In [61]:
null_cities_query = "\
SELECT facts.name AS country, cities.population \
FROM facts \
LEFT JOIN cities ON cities.facts_id = facts.id \
WHERE cities.facts_id IS NULL;"

pd.read_sql_query(null_cities_query,
                 conn).head()

,country,population
0,Kosovo,None
1,Monaco,None
2,Nauru,None
3,San Marino,None
4,Singapore,None


There are two less-common join types SQLite does not support that you should be aware of. The first is a **right join**. A right join, as the name indicates, is exactly the opposite of a left join. Where the left join includes all rows in the table *before* the JOIN clause, the right join includes all rows in the new table in the JOIN clause.

The following two queries, one using a left join and one using a right join, produce identical results.

```SQL
SELECT f.name country, c.name city
FROM facts f
LEFT JOIN cities c ON c.facts_id = f.id
LIMIT 5;
```

```SQL
SELECT f.name country, c.name city
FROM cities c
RIGHT JOIN facts f ON f.id = c.facts_id
LIMIT 5;
```

The main reason a right join would be used is when you are joining more than two tables. In these cases, using a right join is preferable because it can avoid restructuring your whole query to join one table. Outside of this, right joins are used reasonably rarely, so for simple joins it's better to use a left join than a right as it will be easier for your query to be read and understood by others.

The other join type not supported by SQLite is a **full outer** join. A full outer join will include all rows from the tables on both sides of the join.

Like right joins, full outer joins are reasonably uncommon, and similar results can be achieved using a union clause (which we will teach in the next mission). The standard SQL syntax for an full outer join is:

```SQL
SELECT f.name country, c.name city
FROM cities c
FULL OUTER JOIN facts f ON f.id = c.facts_id
LIMIT 5;
```

When joining cities and facts with a full outer join, the result will be be the same as our left and right joins above, because there are no values in `cities.facts_id` that don't exist in `facts.id`.

Once again, here's the Venn diagrams of each join type side by side:

<img src="https://i.imgur.com/PxKTxj8.png" alt="SQL-joins" width="600"/>

Knowing this, let's write a query that returns the 10 capital cities with the highest population ranked from biggest to smallest population:

```SQL
SELECT cities.name AS capital_city, facts.name AS country, cities.population
FROM cities
INNER JOIN facts ON facts.id = cities.facts_id
WHERE cities.capital = 1
ORDER BY cities.population
LIMIT 10;
```

In [62]:
top_pop_capitals_query = "\
SELECT cities.name AS capital_city, facts.name AS country, cities.population \
FROM cities \
INNER JOIN facts ON facts.id = cities.facts_id \
WHERE cities.capital = 1 \
ORDER BY cities.population \
LIMIT 10;"

pd.read_sql_query(top_pop_capitals_query, conn)

,capital_city,country,population
0,Melekeok,Palau,1000
1,Jamestown,"Saint Helena, Ascension, and Tristan da Cunha",1000
2,Mata-Utu,Wallis and Futuna,1000
3,The Valley,Anguilla,2000
4,Stanley,Falkland Islands (Islas Malvinas),2000
5,Vaduz,Liechtenstein,5000
6,Saint-Pierre,Saint Pierre and Miquelon,5000
7,Funafuti,Tuvalu,5000
8,Palikir,"Micronesia, Federated States of",7000
9,Grand Turk,Turks and Caicos Islands,7000


By the way, there's a handy shortcut we can use in our queries which lets us skip the column names, and instead use the order in which the columns appear in the `SELECT` clause. The queary above can be written like this:

```SQL
SELECT c.name capital_city, f.name country, c.population
FROM facts f
INNER JOIN cities c ON c.facts_id = f.id
WHERE c.capital = 1
ORDER BY 3 DESC
LIMIT 10;
```

## Combining Joins with Subqueries

Subqueries can be used to substitute parts of queries, allowing us to find the answers to more complex questions. We can also join to the result of a subquery, just like we could a table.

Here's an example of a using a join and a subquery to produce a table of countries and their capital cities, like we did earlier.

```SQL
SELECT f.name country, c.name capital_city
FROM facts f
INNER JOIN (
            SELECT * FROM cities
            WHERE capital = 1
            ) c ON c.facts_id = f.id
LIMIT 10;
```

Let's write a similar query to find the capital cities with populations of over 10 million:

```SQL
SELECT c.name capital_city, f.name country, c.population
FROM facts f
INNER JOIN (
            SELECT * FROM cities
            WHERE capital = 1
           ) c ON c.facts_id = f.id
WHERE c.population > 10000000
ORDER BY c.population DESC;
```

In [63]:
over_10kk_capitals_query = "\
SELECT c.name capital_city, f.name country, c.population \
FROM facts f \
INNER JOIN ( \
            SELECT * FROM cities \
            WHERE capital = 1 \
           ) c ON c.facts_id = f.id \
WHERE c.population > 10000000 \
ORDER BY c.population DESC"

pd.read_sql_query(over_10kk_capitals_query, conn)

,capital_city,country,population
0,Tokyo,Japan,37217000
1,New Delhi,India,22654000
2,Mexico City,Mexico,20446000
3,Beijing,China,15594000
4,Dhaka,Bangladesh,15391000
5,Buenos Aires,Argentina,13528000
6,Manila,Philippines,11862000
7,Moscow,Russia,11621000
8,Cairo,Egypt,11169000


Let's take everything we've learned before and use it to write a more complex query. When you're writing complex queries with joins and subqueries, it helps to follow this process:

* Think about what data you need in your final output
* Work out which tables you'll need to join, and whether you will need to join to a subquery
    * If you need to join to a subquery, write the subquery first
* Then start writing your `SELECT` clause, followed by the join and any other clauses you will need
* Don't be afraid to write your query in steps, running it as you go— for instance you can run your subquery as a 'stand alone' query first to make sure it looks like you want before writing the outer query

We will be writing a query to find the countries where the urban center (city) population is more than half of the country's total population.

```SQL
SELECT
    f.name country,
    c.urban_pop,
    f.population total_pop,
    (c.urban_pop / CAST(f.population AS FLOAT)) urban_pct
FROM facts f
INNER JOIN (
            SELECT
                facts_id,
                SUM(population) urban_pop
            FROM cities
            GROUP BY facts_id
           ) c ON c.facts_id = f.id
WHERE urban_pct > .5
ORDER BY 4 ASC;
```

In [64]:
complex_query = "\
SELECT \
    f.name country, \
    c.urban_pop, \
    f.population total_pop, \
    (c.urban_pop / CAST(f.population AS FLOAT)) urban_pct \
FROM facts f \
INNER JOIN ( \
            SELECT \
                facts_id, \
                SUM(population) urban_pop \
            FROM cities \
            GROUP BY facts_id \
           ) c ON c.facts_id = f.id \
WHERE urban_pct > .5 \
ORDER BY 4 ASC;"

pd.read_sql_query(complex_query, conn)

,country,urban_pop,total_pop,urban_pct
0,Uruguay,1672000,3341893,0.500315
1,"Congo, Republic of the",2445000,4755097,0.514185
2,Brunei,241000,429646,0.560927
3,New Caledonia,157000,271615,0.578024
4,Virgin Islands,60000,103574,0.579296
5,Falkland Islands (Islas Malvinas),2000,3361,0.595061
6,Djibouti,496000,828324,0.598800
7,Australia,13789000,22751014,0.606083
8,Iceland,206000,331918,0.620635
9,Israel,5226000,8049314,0.649248


## Larger Databases

We'll learn some new techniques to work with the sort of databases that most businesses will use. We'll be working with a modified version of a database called Chinook. The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables.

Here's a schema diagram for the Chinook database:

<img src="https://s3.amazonaws.com/dq-content/189/chinook-schema.svg" alt="chinook-scheme" width="600"/>

You may need to refer back to the schema diagram. This is normal, so don't be bothered when that happens. It can also be helpful to write a simple query to check the column names and some example data from any of the tables. It may look like this:

```SQL
SELECT * FROM album LIMIT 3;
```

Our first task is to gather some information on a specific purchase. For one single purchase (`invoice_id`) we want to know, for each track purchased:

* The id of the track
* The name of the track
* The name of media type of the track.
* The price that the customer paid for the track
* The quantity of the track that was purchased

To gather this information, we will need to write a query that joins 3 tables: `invoice_line`, `track`, and `media_type`. To make this easier, a schema diagram of just these three tables is below:

<img src="https://s3.amazonaws.com/dq-content/189/schema_il_t_mt.svg" alt="little_scheme" width="600"/>

For each of `invoice_line`, `track`, and `media_type`:
* Write a simple query to view the first few rows of each of these tables
* Look at the result of the query to become familiar with the columns and what sort of data we are working with

Now that you're familiar with the data, we can look at the syntax for joining data from more than 2 tables.

```SQL
SELECT [column_names] FROM [table_name_one]
[join_type] JOIN [table_name_two] ON [join_constraint]
[join_type] JOIN [table_name_three] ON [join_constraint];
```

Joining multiple tables is as simple as adding an extra JOIN clause. The SQL engine interprets joins in order, so the first join will be executed, and then the second join will be executed against the result of the first join.

We will use the `invoice_line` table in our `FROM` clause, since it contains 3 of the 5 columns we want in our final query. You don't have to follow this pattern but if you're not sure, it's a good place to start. Since our tasks involves looking for information about a specific `invoice_id`, let's choose an `invoice_id` value of `4`.

Here's how the query should look like:

```SQL
SELECT 
        il.track_id, 
        t.name track_name, 
        mt.name track_type, 
        il.unit_price, 
        il.quantity 
FROM invoice_line il 
INNER JOIN track t ON t.track_id = il.track_id 
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id 
WHERE invoice_id = 4;
```

In [65]:
conn = sqlite3.connect("chinook.db")

In [66]:
complex_track_query = "\
SELECT \
        il.track_id, \
        t.name track_name, \
        mt.name track_type, \
        il.unit_price, \
        il.quantity \
FROM invoice_line il \
INNER JOIN track t ON t.track_id = il.track_id \
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id \
WHERE invoice_id = 4;"

pd.read_sql_query(complex_track_query, conn)

,track_id,track_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",Protected AAC audio file,0.99,1
1,2560,Violent Pornography,MPEG audio file,0.99,1
2,3336,War Pigs,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,MPEG audio file,0.99,1
4,1872,Attitude,MPEG audio file,0.99,1
5,748,Dealer,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),MPEG audio file,0.99,1
7,2514,Spoonman,MPEG audio file,0.99,1


## Joining More Than Three Tables

Let's extend the query we wrote in the previous screen by adding the artist for each track. If you examine the schema, you'll notice that the data for the artist's name is not directly connected to the `track` table.

In this case, we will need to join two new tables to our existing query:

* `artist`, which contains the artist name data that we need
* `album`, which has a column common to each of the `artist` and `track` tables which allows us to join those two tables

Our select clause won't actually use any of the columns from the `album` table. This is quite common when writing more complex queries to need to join a specific table because it will let you join to another table.

```SQL
SELECT
        il.track_id,
        t.name track_name,
        ar.name artist_name,
        mt.name track_type,
        il.unit_price,
        il.quantity
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON ar.artist_id = al.artist_id
WHERE invoice_id = 4;
```

In [67]:
complex_track_query_v2 = "\
SELECT \
        il.track_id, \
        t.name track_name, \
        ar.name artist_name, \
        mt.name track_type, \
        il.unit_price, \
        il.quantity \
FROM invoice_line il \
INNER JOIN track t ON t.track_id = il.track_id \
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id \
INNER JOIN album al ON al.album_id = t.album_id \
INNER JOIN artist ar ON ar.artist_id = al.artist_id \
WHERE invoice_id = 4;"

pd.read_sql_query(complex_track_query_v2, conn)

,track_id,track_name,artist_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",The King's Singers,Protected AAC audio file,0.99,1
1,2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
2,3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
4,1872,Attitude,Metallica,MPEG audio file,0.99,1
5,748,Dealer,Deep Purple,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
7,2514,Spoonman,Soundgarden,MPEG audio file,0.99,1


## Combining Multiple Joins with Subqueries

Because the `invoice_line` table contains each individual song from each customer purchase, it contains information about which songs are purchased the most. We can use the table to find out which artists are purchased the most. Specifically, we want to produce a query that lists the top 10 artists, calculated by the number of times a track by that artist has been purchased.

We'll need to use a `GROUP BY` clause to get the number of tracks purchased from each artist, but before we do we'll have to join a few tables. As we start thinking about how we will write this query, we can look at the database schema from the first screen of this mission and think about what tables we will need to include.

Writing our query would be a lot easier if we had one table that contained both the `track.track_id` and the `artist.name` columns. We can write a subquery that creates this table for us, and then use that subquery to calculate our answer. This means our process will be:

* Write a subquery that produces a table with `track.track_id` and `artist.name`
* Join that subquery to the `invoice_line` table
* Use a `GROUP BY` statement to calculate the number of times each artist has had a track purchased, and find the top 10

We can write our subquery by joining `album` to `track` and then `artist` to `album`, just like we did in the previous screen. We'll add an `ORDER BY` and `LIMIT` to our query so we're only looking at manageable sample of the data, but we'll remove it when we move to the next step.

```SQL
SELECT
    t.track_id,
    ar.name artist_name
FROM track t
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON ar.artist_id = al.artist_id
ORDER BY 1 
LIMIT 5;
```

In [68]:
pd.read_sql_query("\
SELECT \
    t.track_id, \
    ar.name artist_name \
FROM track t \
INNER JOIN album al ON al.album_id = t.album_id \
INNER JOIN artist ar ON ar.artist_id = al.artist_id \
ORDER BY 1 LIMIT 5;", 
                  conn)

,track_id,artist_name
0,1,AC/DC
1,2,Accept
2,3,Accept
3,4,Accept
4,5,Accept


Next, we need to join this subquery to our `invoice_line` table. We'll give our subquery an alias `ta` for 'track artist' to make it easier to refer to. Again, we'll add an `ORDER BY` and `LIMIT` statement to this step so our output is more manageable.

```SQL
SELECT
    il.invoice_line_id,
    il.track_id,
    ta.artist_name
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
ORDER BY 1 LIMIT 5;
```

In [69]:
pd.read_sql_query("\
SELECT \
    il.invoice_line_id, \
    il.track_id, \
    ta.artist_name \
FROM invoice_line il \
INNER JOIN ( \
            SELECT \
                t.track_id, \
                ar.name artist_name \
            FROM track t \
            INNER JOIN album al ON al.album_id = t.album_id \
            INNER JOIN artist ar ON ar.artist_id = al.artist_id \
           ) ta \
           ON ta.track_id = il.track_id \
ORDER BY 1 LIMIT 5;", 
                  conn)

,invoice_line_id,track_id,artist_name
0,1,1158,Guns N' Roses
1,2,1159,Guns N' Roses
2,3,1160,Guns N' Roses
3,4,1161,Guns N' Roses
4,5,1162,Guns N' Roses


At first it might look like we've done something wrong, because the artist for all rows is Guns N' Roses, but that's because the very first order in our table is a customer who purchased an entire Guns N' Roses album! All that remains now is for us to add our `GROUP BY` clause, remove the extra columns and use `ORDER BY` and `LIMIT` clauses to select the 10 most popular artists.

```SQL
SELECT
    ta.artist_name artist,
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1
ORDER BY 2 DESC LIMIT 10;
```

In [70]:
pd.read_sql_query("\
SELECT \
    ta.artist_name artist, \
    COUNT(*) tracks_purchased \
FROM invoice_line il \
INNER JOIN ( \
            SELECT \
                t.track_id, \
                ar.name artist_name \
            FROM track t \
            INNER JOIN album al ON al.album_id = t.album_id \
            INNER JOIN artist ar ON ar.artist_id = al.artist_id \
           ) ta \
           ON ta.track_id = il.track_id \
GROUP BY 1 \
ORDER BY 2 DESC LIMIT 10;",
                 conn)

,artist,tracks_purchased
0,Queen,192
1,Jimi Hendrix,187
2,Nirvana,130
3,Red Hot Chili Peppers,130
4,Pearl Jam,129
5,AC/DC,124
6,Guns N' Roses,124
7,Foo Fighters,121
8,The Rolling Stones,117
9,Metallica,106


We can use a similar technique to work out from which albums the most songs have been purchased.

```SQL
SELECT 
    ta.album_title album, 
    ta.artist_name artist, 
    COUNT(*) tracks_purchased 
FROM invoice_line il 
INNER JOIN ( 
            SELECT 
                t.track_id, 
                ar.name artist_name, 
                al.title album_title 
            FROM track t 
            INNER JOIN album al ON al.album_id = t.album_id 
            INNER JOIN artist ar ON ar.artist_id = al.artist_id 
           ) ta 
           ON ta.track_id = il.track_id 
GROUP BY 1
ORDER BY 3 DESC
LIMIT 5;
```

In [71]:
pd.read_sql_query("\
SELECT \
    ta.album_title album, \
    ta.artist_name artist, \
    COUNT(*) tracks_purchased \
FROM invoice_line il \
INNER JOIN ( \
            SELECT \
                t.track_id, \
                ar.name artist_name, \
                al.title album_title \
            FROM track t \
            INNER JOIN album al ON al.album_id = t.album_id \
            INNER JOIN artist ar ON ar.artist_id = al.artist_id \
           ) ta \
           ON ta.track_id = il.track_id \
GROUP BY 1 \
ORDER BY 3 DESC \
LIMIT 5;",
                 conn)

,album,artist,tracks_purchased
0,Are You Experienced?,Jimi Hendrix,187
1,Faceless,Godsmack,96
2,Mezmerize,System Of A Down,93
3,Get Born,JET,90
4,The Doors,The Doors,83


## Recursive Join

In some cases, there can be a relation between two columns within the same table. We can see that in our `employee` table, where there is a `reports_to` column that has a relation to the `employee_id` column within the same table.

The `reports_to` column identifies each employee's supervisor. If we wanted to create a report of each employee and their supervisor's name, we would need some way of joining a table to itself. Doing this is called a **recursive join**.

Technically, a recursive join will use one of the other standard joins— usually an `INNER JOIN` or `LEFT JOIN` since these are the most commonly used joins— and aliases to distinguish between the table on each side of the join. Here's a simple example of a recursive join in action:

```SQL
SELECT
    e1.employee_id,
    e2.employee_id supervisor_id
FROM employee e1
INNER JOIN employee e2 on e1.reports_to = e2.employee_id
LIMIT 4;
```

In [72]:
pd.read_sql_query("\
SELECT \
    e1.employee_id, \
    e2.employee_id supervisor_id \
FROM employee e1 \
INNER JOIN employee e2 on e1.reports_to = e2.employee_id \
LIMIT 4;",
                 conn)

,employee_id,supervisor_id
0,2,1
1,6,1
2,3,2
3,4,2


To make our report more meaningful, we'll need to add some extra columns. One thing that would be nice is being able to combine the `first_name` and `last_name` columns into a single column. We can do that using the **concatenate operator**: `||`.

You can explore how the concatenate operator works by writing a simple query:

```SQL
SELECT ("this" || "is" || "my" || "string");
```

In [73]:
pd.read_sql_query('SELECT ("this" || "is" || "my" || "string");', conn)

,"(""this"" || ""is"" || ""my"" || ""string"")"
0,thisismystring


Here's a different example of using concatenation, this time with columns from the `album` table:

```SQL
SELECT
    album_id,
    artist_id,
    "album id is" || album_id col_1,
    "artist id is" || artist_id col2,
    album_id || artist_id col3
FROM album LIMIT 3;
```

In [74]:
pd.read_sql_query('\
SELECT \
    album_id, \
    artist_id, \
    "album id is" || album_id col_1, \
    "artist id is" || artist_id col2, \
    album_id || artist_id col3 \
FROM album LIMIT 3;', conn)

,album_id,artist_id,col_1,col2,col3
0,1,1,album id is1,artist id is1,11
1,4,1,album id is4,artist id is1,41
2,2,2,album id is2,artist id is2,22


You'll notice that the SQL engine will concatenate multiple columns, columns with a string, and that the SQL engine handles converting different types where needed.

Let's write a query that returns information about each employee and their supervisor:

```SQL
SELECT 
e1.first_name || " " || e1.last_name employee_name, 
e1.title employee_title, 
e2.first_name || " " || e2.last_name supervisor_name, 
e2.title supervisor_title 
FROM employee e1 
LEFT JOIN employee e2 ON e1.reports_to = e2.employee_id 
ORDER BY employee_name
```

In [75]:
pd.read_sql_query('\
SELECT \
e1.first_name || " " || e1.last_name employee_name, \
e1.title employee_title, \
e2.first_name || " " || e2.last_name supervisor_name, \
e2.title supervisor_title \
FROM employee e1 \
LEFT JOIN employee e2 ON e1.reports_to = e2.employee_id \
ORDER BY employee_name',
                 conn)

,employee_name,employee_title,supervisor_name,supervisor_title
0,Andrew Adams,General Manager,None,None
1,Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
2,Laura Callahan,IT Staff,Michael Mitchell,IT Manager
3,Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Michael Mitchell,IT Manager,Andrew Adams,General Manager
5,Nancy Edwards,Sales Manager,Andrew Adams,General Manager
6,Robert King,IT Staff,Michael Mitchell,IT Manager
7,Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager


## Pattern Matching Using Like

Let's say you're working as a sales support agent for Chinook, and you get back from lunch and see that one of your colleagues has left a phone message on your desk: "Call Jen". Writing a query to get this information out of the database might seem simple at first:


```SQL
SELECT
    first_name,
    last_name,
    phone
FROM customer
WHERE first_name = "Jen";
```

In [76]:
pd.read_sql_query('\
SELECT \
    first_name, \
    last_name, \
    phone \
FROM customer \
WHERE first_name = "Jen";',
                 conn)

,first_name,last_name,phone


No rows were returned by your query! Suddenly you realize — the name in the database might not be exactly "Jen", it might be Jenny, Jennifer, Jenna, or something else. Rather than having to keep trying different combinations, we can use the LIKE operator to find pattern matches. The syntax for `LIKE` is as follows:

```SQL
WHERE [column_name] LIKE "[pattern]"
```

Your pattern should be the substring you want to match for, and one or more `%` characters:

* `%Jen` - will match Jen at the end of a string, eg Sarah-Jen
* `Jen%` - will match Jen at the start of a string, eg Jenny
* `%Jen%` - will match Jen anywhere within the string, eg Kris Jenner

We'll use the last pattern to give ourselves the best chance of a match:

```SQL
SELECT
    first_name,
    last_name,
    phone
FROM customer
WHERE first_name LIKE "%Jen%";
```

In [77]:
pd.read_sql_query('\
SELECT \
    first_name, \
    last_name, \
    phone \
FROM customer \
WHERE first_name LIKE "%Jen%";',
                 conn)

,first_name,last_name,phone
0,Jennifer,Peterson,+1 (604) 688-2255


Perfect - we have found one match and have Jennifer's number so we can return her call.

Keep in mind that in SQLite `LIKE` is case insensitive, so `LIKE "%jen%"` will match `Jen` and `JEN` and `JeN`. Other flavors of SQL may be case sensitive, so you may need to [use the `LOWER()` function to get a case insensitive match](https://stackoverflow.com/questions/18853452/sql-select-like-insensitive-casing).

Let's use LIKE to find a different customer from our database:

```SQL
SELECT 
first_name, 
last_name, 
phone 
FROM customer 
WHERE LOWER(first_name) LIKE "%belle%";
```

In [78]:
pd.read_sql_query('\
SELECT \
first_name, \
last_name, \
phone \
FROM customer \
WHERE LOWER(first_name) LIKE "%belle%"',
                 conn)

,first_name,last_name,phone
0,Isabelle,Mercier,+33 03 80 73 66 99


## Generating Columns With The Case Statement

Sometimes, you'll want to add some sort of categorization to your query. Let's say we wanted to generate a report summarizing each customer purchases. It might be nice to create a column that puts each customer into a category:

* `small spender` - If the customer's total purchases are less than `$40`
* `big spender` - If the customer's total purchases are greater than `$100`
* `regular` - If the customer's total purchases are between `$40` and `$100` (inclusive)
To achieve this, we'll need to use the case statement. The case statement acts like a series of if/then options for a new column. The syntax for `CASE` is:

```SQL
CASE
    WHEN [comparison_1] THEN [value_1]
    WHEN [comparison_2] THEN [value_2]
    ELSE [value_3]
    END
    AS [new_column_name]
```

There can be 1 or more `WHEN` lines, and the `ELSE` line is optional — without it, rows that don't match any `WHEN` will be assigned a null value. Let's start by looking at a simple example. The `media_type` table contains the name of five different media types:

```SQL
SELECT * FROM media_type;
```

In [79]:
pd.read_sql_query('SELECT * FROM media_type;', conn)

,media_type_id,name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


Let's look at how we can use `CASE` to add a new column protected, which indicates whether each media type is protected.

```SQL
SELECT
    media_type_id,
    name,
    CASE
        WHEN name LIKE '%Protected%' THEN 1
        ELSE 0
        END
        AS protected
FROM media_type;
```

In [80]:
pd.read_sql_query("\
SELECT \
    media_type_id, \
    name, \
    CASE \
        WHEN name LIKE '%Protected%' THEN 1 \
        ELSE 0 \
        END \
        AS protected \
FROM media_type;",
                 conn)

,media_type_id,name,protected
0,1,MPEG audio file,0
1,2,Protected AAC audio file,1
2,3,Protected MPEG-4 video file,1
3,4,Purchased AAC audio file,0
4,5,AAC audio file,0


In this example, our `CASE` statement has a single `WHEN` which looks for a partial match of the string `Protected` in the name column. Any rows with a match get a value of `1`, all other rows get `0`.

Let's now put `CASE` into practice using our initial scenario, where we want to categorize customers by their purchase history.

You'll need to know that you can't use aliases in the `WHEN` line, so when writing a `CASE` statement that deals with aggregate functions, you will need to include the aggregate function in each `WHEN` line:

```SQL
SELECT 
    c.first_name || " " || c.last_name customer_name, 
    COUNT(i.invoice_id) number_of_purchases, 
    SUM(i.total) total_spent, 
    CASE 
        WHEN SUM(i.total) < 40 THEN "small spender"
        WHEN SUM(i.total) > 100 THEN "big spender"
        ELSE "regular"
        END
        AS customer_category 
FROM customer c 
INNER JOIN invoice i ON i.customer_id = c.customer_id 
GROUP BY customer_name 
ORDER BY customer_name;
```

In [81]:
pd.read_sql_query('\
SELECT \
    c.first_name || " " || c.last_name customer_name, \
    COUNT(i.invoice_id) number_of_purchases, \
    SUM(i.total) total_spent, \
    CASE \
        WHEN SUM(i.total) < 40 THEN "small spender" \
        WHEN SUM(i.total) > 100 THEN "big spender" \
        ELSE "regular" \
        END \
        AS customer_category \
FROM customer c \
INNER JOIN invoice i ON i.customer_id = c.customer_id \
GROUP BY customer_name \
ORDER BY customer_name;',
                 conn)

,customer_name,number_of_purchases,total_spent,customer_category
0,Aaron Mitchell,8,70.29,regular
1,Alexandre Rocha,10,69.30,regular
2,Astrid Gruber,9,69.30,regular
3,Bjørn Hansen,9,72.27,regular
4,Camille Bernard,9,79.20,regular
5,Daan Peeters,7,60.39,regular
6,Dan Miller,12,95.04,regular
7,Diego Gutiérrez,5,39.60,small spender
8,Dominique Lefebvre,9,72.27,regular
9,Eduardo Martins,12,60.39,regular


## Building and Organizing Complex Queries

Queries can quickly get visually complex. Taking the time to write your queries to be more easily understood will take a little extra time now, but will save you time when you come back to old queries that you have written, and help your colleagues when you're working in a data team.

One obvious area when it comes to writing queries is the use of **capitalization and whitespace**. Because white space doesn't have any meaning in SQL, it can be used to help convey meaning in a complex query. Let's compare the same query written twice — first without whitespace and capitalization:

```SQL
select ta.artist_name artist, count(*) tracks_sold from invoice_line il
inner join (select t.track_id, ar.name artist_name from track t
inner join album al on al.album_id = t.album_id
inner join artist ar on ar.artist_id = al.artist_id) ta
on ta.track_id = il.track_id group by 1 order by 2 desc limit 10;
select ta.artist_name artist, count(*) tracks_sold from invoice_line il
inner join (select t.track_id, ar.name artist_name from track t
inner join album al on al.album_id = t.album_id
inner join artist ar on ar.artist_id = al.artist_id) ta
on ta.track_id = il.track_id group by 1 order by 2 desc limit 10;
```

And now, with whitespace and capitalization:

```SQL
SELECT
    ta.artist_name artist,
    COUNT(*) tracks_sold
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1
ORDER BY 2 DESC LIMIT 10;
```

As you can see, a little time put into whitespace and capitalization pays off. A few tips to help make your queries more readable:

* If a select statement has more than one column, put each on a new line, indented from the select statement
* Always capitalize SQL function names and keywords
* Put each clause of your query on a new line
* Use indenting to make subqueries appear logically separate

Another important consideration when writing readable queries is the use of **alias names and shortcuts**. Name aliases should be clear – a common convention is using the first letter of the table name, however if you feel that a query is complex you should consider using more explicit aliases. Similarly, at times lines like `GROUP BY 1` can be confusing, and explicitly naming the column will make your query more readable.

If you work in a team, you might consider a SQL style guide — a great guide is available at [SQL style guide](http://www.sqlstyle.guide/), but remember is that readability is more important than consistency. If you have a complex query and you think breaking the style guide will make it more readable, you should do it.

## The With Clause

When constructing complex queries, it's useful to create an intermediate table to produce our final results. You can use subqueries to create these intermediate tables. Unfortunately, the way subqueries are written makes it harder to read — the person reading the query needs to find the subquery and read from the inside-out.

One way to alleviate this is to use a **with clause**. With clauses allow you to define one or more named subqueries before the start of the main query. The main query then refers to the subquery by it's alias name, just as if it's a table in the database.

The syntax for the `WITH` clause is relatively straight-forward.

```SQL
WITH [alias_name] AS ([subquery])

SELECT [main_query]
```

Let's look at a simple example, a query designed to gather some info about the tracks from a single album. First, here's our query written with a standard subquery and **no** `WITH` clause:

```SQL
SELECT * FROM
    (
     SELECT
         t.name,
         ar.name artist,
         al.title album_name,
         mt.name media_type,
         g.name genre,
         t.milliseconds length_milliseconds
     FROM track t
     INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
     INNER JOIN genre g ON g.genre_id = t.genre_id
     INNER JOIN album al ON al.album_id = t.album_id
     INNER JOIN artist ar ON ar.artist_id = al.artist_id
    )
WHERE album_name = "Jagged Little Pill";
```

By moving the subquery before the main query using a `WITH` clause, the intent of the main query becomes much easier to understand.

```SQL
WITH track_info AS
    (                
     SELECT
         t.name,
         ar.name artist,
         al.title album_name,
         mt.name media_type,
         g.name genre,
         t.milliseconds length_milliseconds
     FROM track t
     INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
     INNER JOIN genre g ON g.genre_id = t.genre_id
     INNER JOIN album al ON al.album_id = t.album_id
     INNER JOIN artist ar ON ar.artist_id = al.artist_id
    )

SELECT * FROM track_info
WHERE album_name = "Jagged Little Pill";
```

While in this example the difference is subtle, using the `WITH` statement helps a lot when your main query even has some slight complexities. Let's get some practice using `WITH` in a more complex example:

```SQL
WITH playlist_info AS 
    ( 
     SELECT 
        pl.playlist_id, 
        pl.name playlist_name, 
        t.name track_name, 
        t.milliseconds / 1000 length_seconds 
        FROM playlist pl 
        LEFT JOIN playlist_track plt ON plt.playlist_id = pl.playlist_id 
        LEFT JOIN track t ON t.track_id = plt.track_id 
    ) 

SELECT 
    playlist_id, 
    playlist_name, 
    COUNT(track_name) number_of_tracks, 
    SUM(length_seconds) length_seconds 
FROM playlist_info 
GROUP BY playlist_id, playlist_name 
ORDER BY playlist_id;
```

In [82]:
pd.read_sql_query("\
WITH playlist_info AS \
    ( \
     SELECT \
        pl.playlist_id, \
        pl.name playlist_name, \
        t.name track_name, \
        t.milliseconds / 1000 length_seconds \
        FROM playlist pl \
        LEFT JOIN playlist_track plt ON plt.playlist_id = pl.playlist_id \
        LEFT JOIN track t ON t.track_id = plt.track_id \
    ) \
\
SELECT \
    playlist_id, \
    playlist_name, \
    COUNT(track_name) number_of_tracks, \
    SUM(length_seconds) length_seconds \
FROM playlist_info \
GROUP BY playlist_id, playlist_name \
ORDER BY playlist_id;", conn)

,playlist_id,playlist_name,number_of_tracks,length_seconds
0,1,Music,3290,876049.0
1,2,Movies,0,NaN
2,3,TV Shows,213,500987.0
3,4,Audiobooks,0,NaN
4,5,90’s Music,1477,397970.0
5,6,Audiobooks,0,NaN
6,7,Movies,0,NaN
7,8,Music,3290,876049.0
8,9,Music Videos,1,294.0
9,10,TV Shows,213,500987.0


## Creating Views

When we use the `WITH` clause, we're creating a temporary named subquery that we can use only within that query. But what if we find ourselves using the same `WITH` with lots of different queries? It would be nice to permanently define a subquery that we can use again and again.

We do this by creating a **view**, which we can then use in all future queries. An easy way to think of this is the `WITH` clause creates a temporary view. The syntax for creating a view is:

```SQL
CREATE VIEW view_name AS
    SELECT * FROM table;
```

Here's an example of how to create a view called `customer_2`, identical to the existing customer table:

```SQL
CREATE VIEW customer_2 AS
    SELECT * FROM customer;
```

If we wanted to modify this view, and tried to redefine it, we'd get an error:

```SQL
CREATE VIEW customer_2 AS
    SELECT
        customer_id,
        first_name || last_name name,
        phone,
        email,
        support_rep_id
    FROM customer;
```
```
Error: table customer_2 already exists
```

If we wish to redefine a view, we first have to delete, or drop the existing view:

```SQL
DROP VIEW customer_2;
```

We're going to create two views that give us versions of the `customer` table where the customers in the view have specific criteria. The first is a view of all customers that live in the USA.

```SQL
CREATE VIEW customer_usa AS 
     SELECT * FROM customer
     WHERE country = "USA";
```

In [89]:
conn.execute('\
CREATE VIEW customer_usa AS \
     SELECT * FROM customer \
     WHERE country = "USA";')

By the way! You can execute a query using `Connection.execute()` in python sqlite3.

Once a view is created it acts exactly like a table - you don't need to specify that it's a view when you are querying it, and you can do anything with a view that you could do with a table.

Let's create a second view of customers that have purchased more than $90 from our store:

```SQL
CREATE VIEW chinook.customer_gt_90_dollars AS 
    SELECT 
        c.* 
    FROM chinook.customer c 
    LEFT JOIN chinook.invoice i ON i.customer_id = c.customer_id 
    GROUP BY c.customer_id 
    HAVING SUM(i.total) > 90; 

SELECT * FROM chinook.customer_gt_90_dollars;
```

In [94]:
conn.execute('CREATE VIEW customer_gt_90_dollars AS \
    SELECT \
        c.* \
    FROM customer c \
    LEFT JOIN invoice i ON i.customer_id = c.customer_id \
    GROUP BY c.customer_id \
    HAVING SUM(i.total) > 90;')
pd.read_sql_query('SELECT * FROM customer_gt_90_dollars;', conn, index_col = 'customer_id')

,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
customer_id,,,,,,,,,,,,
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4


## Combining Rows With Union

We have now created two views: `customer_usa` and `customer_gt_90_dollars`. How can we find customers who are in different permutations of these two views:

* Customers in the USA **or have** spent more than \$90
* Customers in the USA **and have** spent more than \$90
* Customers in the USA **and have not** spent more than \$90

These scenarios require a different type of join as we're wanting to join rows from not tables, and not columns. Let's start by looking at just the first scenario, where we want to combine rows that exist in either view.

Where regular joins are used to join columns, the **union** operator is used to join rows from tables and/or views.

The syntax for the union operator is composed of two or more `SELECT` statements:

```SQL
[select_statement_one]
UNION
[select_statement_two]
```

Rather than using the `ON` keyword, the statements before and after `UNION` must have the same number of columns, with compatible types in order. We'll learn more about types later, but as an example, `FLOAT` and `INT` are compatible types, but `FLOAT` and `TEXT` are not.

Because we created `customer_usa` and `customer_gt_90_dollars` with identical column names, order, and type as customer, we can safely use `UNION`.

To achieve our first scenario (identify customers who are in the USA **or have** spent more than \$90), the two `SELECT` statements will be very simple - we can just select all columns and rows from each of the two views.

```SQL
SELECT * FROM chinook.customer_usa 
UNION 
SELECT * FROM chinook.customer_gt_90_dollars;
```

In [95]:
pd.read_sql_query('\
SELECT * FROM customer_usa \
UNION \
SELECT * FROM customer_gt_90_dollars', conn)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
5,16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
6,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
7,18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
8,19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
9,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4


## Combining Rows Using Intersect and Except

The three scenarios we discussed earlier were:

* Customers who are in the USA **or have** spent more than \$90
* Customers who are in the USA **and have** spent more than \$90
* Customers who are in the USA **and have not** spent more than \$90

We just successfully used `UNION` for the first, but what about the other two? There are two other operators that will help us with these - **intersect** and **except**. Combined, these three operators allow us to perform set operations in SQL. Here's a diagram and explanation of how these compare with union.

<img src="https://s3.amazonaws.com/dq-content/190/set_operations.svg" alt="unions" width="600"/>

This means that identifying customers who are in the USA and have spent more than \$90 can be done with the following query:

```SQL
SELECT * from customer_usa

INTERSECT

SELECT * from customer_gt_90_dollars;
```

Identifying customers who are in the USA and have not spent \$90 can be done with the following query:

```SQL
SELECT * from customer_usa

EXCEPT

SELECT * from customer_gt_90_dollars;
```

The results of `UNION`, `INTERSECT` and `EXCEPT` conform to the 'everything in SQL is a table'. The results of these operations can be used in subqueries and joined to other tables for more complex analysis. Let's look at a scenario where we'll need to join the results of a set operation to another table:

```SQL
WITH customers_usa_gt_90 AS 
    ( 
     SELECT * FROM customer_usa 

     INTERSECT 

     SELECT * FROM customer_gt_90_dollars 
    ) 

SELECT 
    e.first_name || " " || e.last_name employee_name, 
    COUNT(c.customer_id) customers_usa_gt_90 
FROM employee e 
LEFT JOIN customers_usa_gt_90 c ON c.support_rep_id = e.employee_id 
WHERE e.title = 'Sales Support Agent' 
GROUP BY employee_name 
ORDER BY employee_name;
```

In [97]:
pd.read_sql_query('\
WITH customers_usa_gt_90 AS \
    ( \
     SELECT * FROM customer_usa \
\
     INTERSECT \
\
     SELECT * FROM customer_gt_90_dollars \
    ) \
\
SELECT \
    e.first_name || " " || e.last_name employee_name, \
    COUNT(c.customer_id) customers_usa_gt_90 \
FROM employee e \
LEFT JOIN customers_usa_gt_90 c ON c.support_rep_id = e.employee_id \
WHERE e.title = "Sales Support Agent" \
GROUP BY employee_name \
ORDER BY employee_name;',
                  conn)

,employee_name,customers_usa_gt_90
0,Jane Peacock,0
1,Margaret Park,2
2,Steve Johnson,2


## Multiple Named Subqueries

Here's the syntax for creating more than one named subquery. To do this, you use a single `WITH` clause and multiple, comma-separated alias/subquery pairs:

```SQL
WITH
    [alias_name] AS ([subquery]),
    [alias_name_2] AS ([subquery_2]),
    [alias_name_3] AS ([subquery_3])

SELECT [main_query]
```

While each subquery can be independent, we can actually use the result of the first subquery in subsequent subqueries, and so on. This can be a useful way of building readable complex queries.

Let's look at a simple example where we create three named subqueries that build on each other.

```SQL
WITH
    usa AS
        (
        SELECT * FROM customer
        WHERE country = "USA"
        ),
    last_name_g AS
        (
         SELECT * FROM usa
         WHERE last_name LIKE "G%"
        ),
    state_ca AS
        (
        SELECT * FROM last_name_g
        WHERE state = "CA"
        )

SELECT
    first_name,
    last_name,
    country,
    state
FROM state_ca
```

In [98]:
pd.read_sql_query('\
WITH \
    usa AS \
        ( \
        SELECT * FROM customer \
        WHERE country = "USA" \
        ), \
    last_name_g AS \
        ( \
         SELECT * FROM usa \
         WHERE last_name LIKE "G%" \
        ), \
    state_ca AS \
        ( \
        SELECT * FROM last_name_g \
        WHERE state = "CA" \
        ) \
\
SELECT \
    first_name, \
    last_name, \
    country, \
    state \
FROM state_ca',
                 conn)

,first_name,last_name,country,state
0,Tim,Goyer,USA,CA


In reality, we'd usually write this as a single query using multiple `AND` operators in our `WHERE` clause, but it helps us demonstrate how multiple subqueries can be defined with a single `WITH` clause. Let's use a more 'real life' example to gather total sales data on customers from India.

```SQL
WITH
    customers_india AS
        (
        SELECT * FROM customer
        WHERE country = "India"
        ),
    sales_per_customer AS
        (
         SELECT
             customer_id,
             SUM(total) total
         FROM invoice
         GROUP BY 1
        )

SELECT
    ci.first_name || " " || ci.last_name customer_name,
    spc.total total_purchases
FROM customers_india ci
INNER JOIN sales_per_customer spc ON ci.customer_id = spc.customer_id
ORDER BY customer_name;
```

In [101]:
pd.read_sql_query('\
WITH \
    customers_india AS \
        ( \
        SELECT * FROM customer \
        WHERE country = "India" \
        ), \
    sales_per_customer AS \
        ( \
         SELECT \
             customer_id, \
             SUM(total) total \
         FROM invoice \
         GROUP BY 1 \
        ) \
\
SELECT \
    ci.first_name || " " || ci.last_name customer_name, \
    spc.total total_purchases \
FROM customers_india ci \
INNER JOIN sales_per_customer spc ON ci.customer_id = spc.customer_id \
ORDER BY customer_name;',
                 conn)

,customer_name,total_purchases
0,Manoj Pareek,111.87
1,Puja Srivastava,71.28


## Wrapping Up

Now, it's time to bring everything we've learned so far to write a complex query! **We will be writing a query to find the customer from each country that has spent the most money at our store.**

```SQL
WITH top_customers AS 
   ( 
    SELECT 
    c.country, 
    c.first_name || " " || c.last_name customer_name, 
    SUM(i.total) total_purchased 
FROM customer c 
INNER JOIN invoice i ON c.customer_id = i.customer_id 
GROUP BY country, customer_name
   ) 
SELECT 
    country,
    customer_name,
    MAX(total_purchased) total_purchased 
FROM top_customers 
GROUP BY country
ORDER BY country;
```

In [103]:
pd.read_sql_query('\
WITH top_customers AS \
   ( \
    SELECT \
    c.country, \
    c.first_name || " " || c.last_name customer_name, \
    SUM(i.total) total_purchased \
FROM customer c \
INNER JOIN invoice i ON c.customer_id = i.customer_id \
GROUP BY country, customer_name \
   ) \
SELECT \
    country, \
    customer_name, \
    MAX(total_purchased) total_purchased \
FROM top_customers \
GROUP BY country \
ORDER BY country',
                 conn)

,country,customer_name,total_purchased
0,Argentina,Diego Gutiérrez,39.60
1,Australia,Mark Taylor,81.18
2,Austria,Astrid Gruber,69.30
3,Belgium,Daan Peeters,60.39
4,Brazil,Luís Gonçalves,108.90
5,Canada,François Tremblay,99.99
6,Chile,Luis Rojas,97.02
7,Czech Republic,František Wichterlová,144.54
8,Denmark,Kara Nielsen,37.62
9,Finland,Terhi Hämäläinen,79.20
